# Import Library

In [515]:
import pandas as pd
import numpy as np
import re
from pymongo import MongoClient
import sys
import os
import ast
from tabulate import tabulate
import torch

code_path = os.path.abspath('../')
sys.path.append(code_path)

from credentials import connection_string

In [8]:
# Check if CUDA is available
if torch.cuda.is_available():
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No CUDA devices available.")

Number of CUDA devices: 1
Device 0: NVIDIA GeForce RTX 4060 Laptop GPU


# Import Data

## Connect to MongoDB Cluster

In [464]:
database_name = "Singaraja"
collection_name = f"{database_name}_Raw"

In [465]:
client = MongoClient(connection_string)
db = client[database_name]
collection = db[collection_name]

## Get All Documents from Selected Collection

In [466]:
all_documents = collection.find()

## Convert to DataFrame

In [467]:
df = pd.DataFrame(list(all_documents))

# drop _id
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

In [468]:
df

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan
0,Minutasi,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"Kamis, 08 Agu. 2024",41 Hari,"['KETUT SUBAGIA', 'KOMANG ADITYA']","['Kadek Adi Pramarta, S.H.']","['I Made Bagiarta~Hakim Ketua~Ya', 'Anak Agung...",3,['KOMANG ADITYA~Pidana Penjara Waktu Tertentu ...,MENGADILI:\nMenyatakan Terdakwa I. Ketut Subag...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
1,Minutasi,118/Pid.Sus/2024/PN Sgr,Narkotika,"Senin, 05 Agu. 2024",44 Hari,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']","['I Made Bagiarta~Hakim Ketua~Tidak', 'Ni Made...",3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,MENGADILI:\nMenyatakan Terdakwa I NYOMAN SUART...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
2,Minutasi,114/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",48 Hari,['RICO JAYADI'],"['I MADE SUTAPA,S.H.']","['I Made Bagiarta~Hakim Ketua~Ya', 'I Gusti Ay...",3,['RICO JAYADI~Subsider Penjara (3 Bulan )\nPid...,MENGADILI:\nMenyatakan Terdakwa Rico Jayadi te...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
3,Minutasi,115/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",53 Hari,['SANG PUTU WIDIANA'],"['Isnarti Jayaningsih, S.H.']","['I Gusti Made Juliartawan~Hakim Ketua~Tidak',...",4,['SANG PUTU WIDIANA~Subsider Penjara (2 Bulan ...,MENGADILI:\nMenyatakan Terdakwa SANG PUTU WIDI...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
4,Minutasi,116/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",53 Hari,['WAYAN DARNA MASTONO'],"['Isnarti Jayaningsih, S.H.']","['I Gusti Made Juliartawan~Hakim Ketua~Tidak',...",4,['WAYAN DARNA MASTONO~Subsider Penjara (2 Bula...,MENGADILI:\nMenyatakan Terdakwa WAYAN DARNA MA...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
...,...,...,...,...,...,...,...,...,...,...,...,...
1265,Minutasi,223/Pid.Sus/2017/PN Sgr,Perlindungan dan Pengelolaan Lingkungan Hidup,"Kamis, 07 Des. 2017",60 Hari,[Jumat Ariyanto],"[I Nyoman Sulitra, SH.,MH.]","[Ida Bagus Bama Dewa. P~Hakim Ketua~Ya, I Made...",3,[Jumat Ariyanto~Pidana Kurungan (1 Tahun )\nSu...,M E N G A D I L I\nÂ \nMenyatakan Terdakwa Jum...,Kesatu : Pasal 83 ayat (1) huruf b UU RI No. 1...
1266,Minutasi,224/Pid.B/2017/PN Sgr,Kejahatan Perjudian,"Kamis, 07 Des. 2017",48 Hari,[I KETUT ANA SAPUTRA Alias BENCUT],"[MADE ASTINI, SH.]","[Sudar~Hakim Ketua~Ya, Ni Luh Suantini~Hakim A...",1,[I KETUT ANA SAPUTRA Alias BENCUT~Pidana Penja...,Menyatakan Terdakwa I Ketut Ana Saputra Alias ...,Perbuatan terdakwa sebagaimana diatur dan dian...
1267,Minutasi,220/Pid.B/2017/PN Sgr,Penganiayaan,"Rabu, 06 Des. 2017",57 Hari,[Sahabudin Alias Udin],"[I KETUT KINDRA, SH]","[Ni Luh Suantini~Hakim Ketua~Ya, I Gede Karang...",2,[Sahabudin Alias Udin~Pidana Penjara Waktu Ter...,M E N G A D I L I :\nÂ \n1. Menyatakan terdakw...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...
1268,Minutasi,219/Pid.B/2017/PN Sgr,Kejahatan yang Membahayakan Keamananan Umum Ba...,"Rabu, 06 Des. 2017",91 Hari,[Putu Kristian Damanta],"[I GUSTI ARYA WIDNYANA, SH.]","[A.A. Sagung Yuni Wulantrisna~Hakim Ketua~Ya, ...",4,[Putu Kristian Damanta~Pidana Penjara Waktu Te...,"1. Menyatakan Terdakwa Putu Kristian Damanta ,...",Pertama : Pasal 187 ayat (1) KUHP\ndan\nKedua ...


In [469]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1270 entries, 0 to 1269
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   status_perkara       1270 non-null   object
 1   nomor_perkara        1270 non-null   object
 2   klasifikasi_perkara  1270 non-null   object
 3   tanggal_pendaftaran  1270 non-null   object
 4   lama_proses          1270 non-null   object
 5   terdakwa             1270 non-null   object
 6   penuntut_umum        1270 non-null   object
 7   hakim                1270 non-null   object
 8   jumlah_saksi         1270 non-null   int64 
 9   putusan_hukuman      1270 non-null   object
 10  barang_bukti         1270 non-null   object
 11  dakwaan              1267 non-null   object
dtypes: int64(1), object(11)
memory usage: 119.2+ KB


In [470]:

print(df[df['nomor_perkara'] == '86/Pid.B/2019/PN Sgr']['barang_bukti'])


937    M E N G A D I L I :\nMenyatakan Terdakwa NYOMA...
Name: barang_bukti, dtype: object


# Data Understanding

In [ ]:
df.head(5)

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan
0,Minutasi,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"Kamis, 08 Agu. 2024",41 Hari,"['KETUT SUBAGIA', 'KOMANG ADITYA']","['Kadek Adi Pramarta, S.H.']","['I Made Bagiarta~Hakim Ketua~Ya', 'Anak Agung...",3,['KOMANG ADITYA~Pidana Penjara Waktu Tertentu ...,MENGADILI:\nMenyatakan Terdakwa I. Ketut Subag...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
1,Minutasi,118/Pid.Sus/2024/PN Sgr,Narkotika,"Senin, 05 Agu. 2024",44 Hari,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']","['I Made Bagiarta~Hakim Ketua~Tidak', 'Ni Made...",3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,MENGADILI:\nMenyatakan Terdakwa I NYOMAN SUART...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
2,Minutasi,114/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",48 Hari,['RICO JAYADI'],"['I MADE SUTAPA,S.H.']","['I Made Bagiarta~Hakim Ketua~Ya', 'I Gusti Ay...",3,['RICO JAYADI~Subsider Penjara (3 Bulan )\nPid...,MENGADILI:\nMenyatakan Terdakwa Rico Jayadi te...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
3,Minutasi,115/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",53 Hari,['SANG PUTU WIDIANA'],"['Isnarti Jayaningsih, S.H.']","['I Gusti Made Juliartawan~Hakim Ketua~Tidak',...",4,['SANG PUTU WIDIANA~Subsider Penjara (2 Bulan ...,MENGADILI:\nMenyatakan Terdakwa SANG PUTU WIDI...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
4,Minutasi,116/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",53 Hari,['WAYAN DARNA MASTONO'],"['Isnarti Jayaningsih, S.H.']","['I Gusti Made Juliartawan~Hakim Ketua~Tidak',...",4,['WAYAN DARNA MASTONO~Subsider Penjara (2 Bula...,MENGADILI:\nMenyatakan Terdakwa WAYAN DARNA MA...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...


**Dataset Features**
| Fitur | Deskripsi | Tipe Data |
|-----------------------|-----------------------------------------------------------------------|:------------:|
| `status_perkara` | Status dari perkara.| `str` |
| `nomor_perkara` | Nomor identifikasi unik untuk perkara.| `str` |
| `klasifikasi_perkara` | Kategori atau jenis perkara.| `str` |
| `tanggal_pendaftaran` | Tanggal ketika perkara didaftarkan di pengadilan.| `str` |
| `lama_proses` | Durasi waktu yang dibutuhkan untuk memproses perkara.| `str` |
| `terdakwa` | Nama individu yang didakwa dalam perkara tersebut.| `list[str]` |
| `penuntut_umum` | Nama jaksa atau penuntut yang menangani perkara.| `list[str]`  |
| `hakim` | Nama hakim yang memimpin sidang perkara.| `list[str]` |
| `jumlah_saksi` | Total jumlah saksi yang dihadirkan dalam sidang.| `int` |
| `putusan_hukuman` | Putusan atau keputusan yang dijatuhkan oleh hakim.| `list[str]` |
| `barang_bukti` | Amar Putusan yang berisi tentang barang bukti perkara.| `str` |
| `dakwaan`| Uraian mengenai tuduhan atau dakwaan yang diajukan terhadap terdakwa.| `str` |







# Data Preprocessing

## Data Cleaning

### Data Cleaning: Null Values

In [240]:
# Check for null values in the entire DataFrame
null_counts = df.isnull().sum()

print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara         0
nomor_perkara          0
klasifikasi_perkara    0
tanggal_pendaftaran    0
lama_proses            0
terdakwa               0
penuntut_umum          0
hakim                  0
jumlah_saksi           0
putusan_hukuman        0
barang_bukti           0
dakwaan                3
dtype: int64


In [241]:
# Print rows with any null values
null_rows = df[df.isnull().any(axis=1)]

print("Rows with null values:")
print(null_rows)


Rows with null values:
    status_perkara            nomor_perkara klasifikasi_perkara  \
107       Minutasi    151/Pid.B/2023/PN Sgr        Penganiayaan   
135       Minutasi  116/Pid.Sus/2023/PN Sgr           Narkotika   
145       Minutasi    101/Pid.B/2023/PN Sgr         Penggelapan   

      tanggal_pendaftaran lama_proses                                terdakwa  \
107   Jumat, 08 Des. 2023     70 Hari               ['NENGAH IRPAN RUSMAWAN']   
135  Selasa, 17 Okt. 2023     58 Hari  ['K. DEKY CHRISTINA ARTHA Alias DEKY']   
145   Senin, 02 Okt. 2023     59 Hari     ['GEDE SUMARDANA PUTRA alias DESU']   

                                 penuntut_umum  \
107                  ['GEDE PUTU ASTAWA,S.H.']   
135  ['I MADE HERI PERMANA PUTRA, S.H., M.H.']   
145                  ['MADE JUNI ARTINI.S.H.']   

                                                 hakim  jumlah_saksi  \
107  ['Heriyanti~Hakim Ketua~Ya', 'Ni Made Kushanda...             6   
135  ['Heriyanti~Hakim Ketua~Ya', 'I G

In [242]:
df_copy = df.dropna()
# Check for null values in the entire DataFrame
null_counts = df_copy.isnull().sum()

print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara         0
nomor_perkara          0
klasifikasi_perkara    0
tanggal_pendaftaran    0
lama_proses            0
terdakwa               0
penuntut_umum          0
hakim                  0
jumlah_saksi           0
putusan_hukuman        0
barang_bukti           0
dakwaan                0
dtype: int64


### Data Cleaning: Data Disamarkan

In [243]:
count_disamarkan = df_copy[df_copy["dakwaan"].str.lower().isin(["disamarkan"])]

# Print the count
print(f"Jumlah data 'Disamarkan': {count_disamarkan.shape[0]}")


Jumlah data 'Disamarkan': 100


In [244]:
print(count_disamarkan)


     status_perkara            nomor_perkara           klasifikasi_perkara  \
36         Minutasi   73/Pid.Sus/2024/PN Sgr             Perlindungan Anak   
44         Minutasi   62/Pid.Sus/2024/PN Sgr             Perlindungan Anak   
67         Minutasi   37/Pid.Sus/2024/PN Sgr             Perlindungan Anak   
76         Minutasi   27/Pid.Sus/2024/PN Sgr             Perlindungan Anak   
102        Minutasi  155/Pid.Sus/2023/PN Sgr             Perlindungan Anak   
...             ...                      ...                           ...   
1116       Minutasi  144/Pid.Sus/2018/PN Sgr  Kekerasan Dalam Rumah Tangga   
1183       Minutasi   64/Pid.Sus/2018/PN Sgr             Perlindungan Anak   
1231       Minutasi   18/Pid.Sus/2018/PN Sgr  Kekerasan Dalam Rumah Tangga   
1244       Minutasi    1/Pid.Sus/2018/PN Sgr             Perlindungan Anak   
1253       Minutasi  241/Pid.Sus/2017/PN Sgr  Kekerasan Dalam Rumah Tangga   

       tanggal_pendaftaran lama_proses        terdakwa  \
36   

In [245]:
df_copy = df_copy[~df_copy["dakwaan"].str.lower().isin(["disamarkan"])]
df_copy.reset_index(drop=True, inplace=True)

In [246]:
count_disamarkan = df_copy[df_copy["dakwaan"].str.lower().isin(["disamarkan"])]

# Print the count
print(f"Jumlah data 'Disamarkan': {count_disamarkan.shape[0]}")

Jumlah data 'Disamarkan': 0


### Data Cleaning: Status Perkara

In [247]:
unique_status_perkara_values = df_copy['status_perkara'].unique()
num_unique_status_perkara = len(unique_status_perkara_values)

print(f"Number of unique values in 'status_perkara': {num_unique_status_perkara}")
print("Unique values:", unique_status_perkara_values)


Number of unique values in 'status_perkara': 1
Unique values: ['Minutasi']


### Data Cleaning: Nomor Perkara

In [248]:
unique_nomor_perkara_values = df_copy['nomor_perkara'].unique()
num_unique_nomor_perkara = len(unique_nomor_perkara_values)

print(f"Jumlah Data: {len(df_copy)}")
print(f"Jumlah Nilai Unique pada Kolom 'nomor_perkara': {num_unique_nomor_perkara}")

Jumlah Data: 1167
Jumlah Nilai Unique pada Kolom 'nomor_perkara': 1167


### Data Cleaning: Klasifikasi Perkara

In [249]:
unique_klasifikasi_perkara_values = df_copy['klasifikasi_perkara'].unique()
num_unique_klasifikasi_perkara = len(unique_klasifikasi_perkara_values)
unique_klasifikasi_counts = df_copy['klasifikasi_perkara'].value_counts()

print(f"Number of unique values in 'status_perkara': {num_unique_klasifikasi_perkara}")
print(unique_klasifikasi_counts)

Number of unique values in 'status_perkara': 36
klasifikasi_perkara
Pencurian                                                                                               332
Narkotika                                                                                               323
Penganiayaan                                                                                            104
Penggelapan                                                                                              88
Lalu Lintas                                                                                              76
Kejahatan Perjudian                                                                                      63
Lain-Lain                                                                                                55
Penipuan                                                                                                 28
Pembunuhan                                                          

In [250]:
df_copy = df_copy[~df_copy["klasifikasi_perkara"].isin(["Lalu Lintas"])]
df_copy.reset_index(drop=True, inplace=True)

In [251]:
unique_klasifikasi_perkara_values = df_copy['klasifikasi_perkara'].unique()
num_unique_klasifikasi_perkara = len(unique_klasifikasi_perkara_values)
unique_klasifikasi_counts = df_copy['klasifikasi_perkara'].value_counts()

print(f"Number of unique values in 'status_perkara': {num_unique_klasifikasi_perkara}")
print(unique_klasifikasi_counts)

Number of unique values in 'status_perkara': 35
klasifikasi_perkara
Pencurian                                                                                               332
Narkotika                                                                                               323
Penganiayaan                                                                                            104
Penggelapan                                                                                              88
Kejahatan Perjudian                                                                                      63
Lain-Lain                                                                                                55
Penipuan                                                                                                 28
Pembunuhan                                                                                               11
Tindak Pidana Senjata Api atau Benda Tajam                          

### Data Cleaning: Hakim

In [252]:
# Count unique data types in the column
unique_types = df_copy['hakim'].apply(type).unique()

# Display the unique types
print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'str'> <class 'list'>]


In [253]:
count_str_hakim = df_copy[df_copy["hakim"].apply(lambda x: isinstance(x, str))].shape[0]
print(f"Jumlah data dengan 'hakim' berupa string: {count_str_hakim}")


Jumlah data dengan 'hakim' berupa string: 157


In [254]:
for index, row in df_copy.iterrows():
    value = row["hakim"]
    
    # If the value is a string representation of a list, convert it back to a list
    if isinstance(value, str):
        try:
            print(f"Type: {type(value)}, Row {index} - Value: {value}")
            value = ast.literal_eval(value)  # Convert the string back to a list
            df_copy.at[index, "hakim"] = value  # Update the value in the DataFrame
            print(f"Type: {type(value)}, Row {index} - Value: {value}\n")
            
        except (ValueError, SyntaxError):
            print(f"Error in Row {index}: Not a valid list format.")
    

Type: <class 'str'>, Row 0 - Value: ['I Made Bagiarta~Hakim Ketua~Ya', 'Anak Agung Ayu Sri Sudanthi~Hakim Anggota~Ya', 'Pulung Yustisia Dewi~Hakim Anggota~Ya']
Type: <class 'list'>, Row 0 - Value: ['I Made Bagiarta~Hakim Ketua~Ya', 'Anak Agung Ayu Sri Sudanthi~Hakim Anggota~Ya', 'Pulung Yustisia Dewi~Hakim Anggota~Ya']

Type: <class 'str'>, Row 1 - Value: ['I Made Bagiarta~Hakim Ketua~Tidak', 'Ni Made Kushandari~Hakim Anggota~Tidak', 'Made Astina Dwipayana~Hakim Anggota~Tidak', 'I Made Bagiarta~Hakim Ketua~Ya', 'Ni Made Kushandari~Hakim Anggota~Ya', 'Ni Putu Asih Yudiastri~Hakim Anggota~Ya']
Type: <class 'list'>, Row 1 - Value: ['I Made Bagiarta~Hakim Ketua~Tidak', 'Ni Made Kushandari~Hakim Anggota~Tidak', 'Made Astina Dwipayana~Hakim Anggota~Tidak', 'I Made Bagiarta~Hakim Ketua~Ya', 'Ni Made Kushandari~Hakim Anggota~Ya', 'Ni Putu Asih Yudiastri~Hakim Anggota~Ya']

Type: <class 'str'>, Row 2 - Value: ['I Made Bagiarta~Hakim Ketua~Ya', 'I Gusti Ayu Kade Ari Wulandari~Hakim Anggota~Ya', 

In [255]:
# Count unique data types in the column
unique_types = df_copy['hakim'].apply(type).unique()

# Display the unique types
print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'list'>]


In [256]:
unique_types = df_copy['hakim'].apply(len).unique()
print(unique_types)

[3 6 9]


In [257]:
for index, row in df_copy.iterrows():
    if len(row["hakim"]) == 6: 
        print(f"{index} -> {row['hakim']}")

1 -> ['I Made Bagiarta~Hakim Ketua~Tidak', 'Ni Made Kushandari~Hakim Anggota~Tidak', 'Made Astina Dwipayana~Hakim Anggota~Tidak', 'I Made Bagiarta~Hakim Ketua~Ya', 'Ni Made Kushandari~Hakim Anggota~Ya', 'Ni Putu Asih Yudiastri~Hakim Anggota~Ya']
3 -> ['I Gusti Made Juliartawan~Hakim Ketua~Tidak', 'Ni Made Kushandari~Hakim Anggota~Tidak', 'Made Astina Dwipayana~Hakim Anggota~Tidak', 'I Gusti Made Juliartawan~Hakim Ketua~Ya', 'Ni Made Kushandari~Hakim Anggota~Ya', 'Ni Putu Asih Yudiastri~Hakim Anggota~Ya']
4 -> ['I Gusti Made Juliartawan~Hakim Ketua~Tidak', 'Ni Made Kushandari~Hakim Anggota~Tidak', 'Made Astina Dwipayana~Hakim Anggota~Tidak', 'I Gusti Made Juliartawan~Hakim Ketua~Ya', 'Ni Made Kushandari~Hakim Anggota~Ya', 'Ni Putu Asih Yudiastri~Hakim Anggota~Ya']
7 -> ['I Made Bagiarta~Hakim Ketua~Tidak', 'I Gusti Made Juliartawan~Hakim Anggota~Tidak', 'Ni Putu Asih Yudiastri~Hakim Anggota~Tidak', 'I Made Bagiarta~Hakim Ketua~Ya', 'I Gusti Made Juliartawan~Hakim Anggota~Ya', 'I Gusti A

In [258]:
for index, row in df_copy.iterrows():
    if len(row["hakim"]) == 9: 
        print(f"{index} -> {row['hakim']}")

537 -> ['I Gede Yuliartha~Hakim Ketua~Tidak', 'A.A. Ayu Merta Dewi~Hakim Anggota~Tidak', 'Anak Agung Ngurah Budhi Dharmawan~Hakim Anggota~Tidak', 'I Gede Yuliartha~Hakim Ketua~Tidak', 'Anak Agung Ngurah Budhi Dharmawan~Hakim Anggota~Tidak', 'Wayan Eka Satria Utama~Hakim Anggota~Tidak', 'I Gede Yuliartha~Hakim Ketua~Ya', 'Anak Agung Ngurah Budhi Dharmawan~Hakim Anggota~Ya', 'I Nyoman Dipa Rudiana~Hakim Anggota~Ya']


In [259]:
for index, row in df_copy.iterrows():
    i = len(row["hakim"])
    for j in range(i):
        print(row["hakim"][j])
    print("")

I Made Bagiarta~Hakim Ketua~Ya
Anak Agung Ayu Sri Sudanthi~Hakim Anggota~Ya
Pulung Yustisia Dewi~Hakim Anggota~Ya

I Made Bagiarta~Hakim Ketua~Tidak
Ni Made Kushandari~Hakim Anggota~Tidak
Made Astina Dwipayana~Hakim Anggota~Tidak
I Made Bagiarta~Hakim Ketua~Ya
Ni Made Kushandari~Hakim Anggota~Ya
Ni Putu Asih Yudiastri~Hakim Anggota~Ya

I Made Bagiarta~Hakim Ketua~Ya
I Gusti Ayu Kade Ari Wulandari~Hakim Anggota~Ya
Pulung Yustisia Dewi~Hakim Anggota~Ya

I Gusti Made Juliartawan~Hakim Ketua~Tidak
Ni Made Kushandari~Hakim Anggota~Tidak
Made Astina Dwipayana~Hakim Anggota~Tidak
I Gusti Made Juliartawan~Hakim Ketua~Ya
Ni Made Kushandari~Hakim Anggota~Ya
Ni Putu Asih Yudiastri~Hakim Anggota~Ya

I Gusti Made Juliartawan~Hakim Ketua~Tidak
Ni Made Kushandari~Hakim Anggota~Tidak
Made Astina Dwipayana~Hakim Anggota~Tidak
I Gusti Made Juliartawan~Hakim Ketua~Ya
Ni Made Kushandari~Hakim Anggota~Ya
Ni Putu Asih Yudiastri~Hakim Anggota~Ya

Made Hermayanti Muliartha~Hakim Ketua~Ya
Anak Agung Ayu Sri Su

In [260]:
df_copy.loc[:, 'hakim'] = df_copy['hakim'].apply(lambda x: [entry for entry in x if '~Tidak' not in entry])

In [261]:
for index, row in df_copy.iterrows():
    i = len(row["hakim"])
    for j in range(i):
        print(row["hakim"][j])
    print("")

I Made Bagiarta~Hakim Ketua~Ya
Anak Agung Ayu Sri Sudanthi~Hakim Anggota~Ya
Pulung Yustisia Dewi~Hakim Anggota~Ya

I Made Bagiarta~Hakim Ketua~Ya
Ni Made Kushandari~Hakim Anggota~Ya
Ni Putu Asih Yudiastri~Hakim Anggota~Ya

I Made Bagiarta~Hakim Ketua~Ya
I Gusti Ayu Kade Ari Wulandari~Hakim Anggota~Ya
Pulung Yustisia Dewi~Hakim Anggota~Ya

I Gusti Made Juliartawan~Hakim Ketua~Ya
Ni Made Kushandari~Hakim Anggota~Ya
Ni Putu Asih Yudiastri~Hakim Anggota~Ya

I Gusti Made Juliartawan~Hakim Ketua~Ya
Ni Made Kushandari~Hakim Anggota~Ya
Ni Putu Asih Yudiastri~Hakim Anggota~Ya

Made Hermayanti Muliartha~Hakim Ketua~Ya
Anak Agung Ayu Sri Sudanthi~Hakim Anggota~Ya
Wayan Eka Satria Utama~Hakim Anggota~Ya

I Gusti Made Juliartawan~Hakim Ketua~Ya
I Gusti Ayu Kade Ari Wulandari~Hakim Anggota~Ya
Made Astina Dwipayana~Hakim Anggota~Ya

I Made Bagiarta~Hakim Ketua~Ya
I Gusti Made Juliartawan~Hakim Anggota~Ya
I Gusti Ayu Kade Ari Wulandari~Hakim Anggota~Ya

I Gusti Ayu Kade Ari Wulandari~Hakim Ketua~Ya
Ma

In [262]:
unique_types = df_copy['hakim'].apply(len).unique()
print(unique_types)

[3]


In [263]:
unique_types = df_copy['hakim'].apply(type).unique()
print(unique_types)

[<class 'list'>]


In [264]:
df_copy.loc[:, 'hakim'] = df_copy['hakim'].apply(lambda x: [value for value in x if '~Hakim Anggota' not in value])

In [265]:
for index, row in df_copy.iterrows():
    print(row['hakim'])

['I Made Bagiarta~Hakim Ketua~Ya']
['I Made Bagiarta~Hakim Ketua~Ya']
['I Made Bagiarta~Hakim Ketua~Ya']
['I Gusti Made Juliartawan~Hakim Ketua~Ya']
['I Gusti Made Juliartawan~Hakim Ketua~Ya']
['Made Hermayanti Muliartha~Hakim Ketua~Ya']
['I Gusti Made Juliartawan~Hakim Ketua~Ya']
['I Made Bagiarta~Hakim Ketua~Ya']
['I Gusti Ayu Kade Ari Wulandari~Hakim Ketua~Ya']
['I Gusti Ayu Kade Ari Wulandari~Hakim Ketua~Ya']
['I Made Bagiarta~Hakim Ketua~Ya']
['Made Hermayanti Muliartha~Hakim Ketua~Ya']
['I Gusti Made Juliartawan~Hakim Ketua~Ya']
['I Gusti Made Juliartawan~Hakim Ketua~Ya']
['I Made Bagiarta~Hakim Ketua~Ya']
['I Made Bagiarta~Hakim Ketua~Ya']
['Made Hermayanti Muliartha~Hakim Ketua~Ya']
['I Gusti Made Juliartawan~Hakim Ketua~Ya']
['I Made Bagiarta~Hakim Ketua~Ya']
['I Gusti Made Juliartawan~Hakim Ketua~Ya']
['I Made Bagiarta~Hakim Ketua~Ya']
['I Made Bagiarta~Hakim Ketua~Ya']
['I Gusti Made Juliartawan~Hakim Ketua~Ya']
['I Gusti Made Juliartawan~Hakim Ketua~Ya']
['I Made Bagiarta~H

In [266]:
unique_types = df_copy['hakim'].apply(len).unique()
print(unique_types)

[1]


In [267]:
df_copy.loc[:, 'hakim'] = df_copy['hakim'].apply(
    lambda x: [value.split('~Hakim')[0].strip() for value in x]
)

In [268]:
for index, row in df_copy.iterrows():
    print(row['hakim'])

['I Made Bagiarta']
['I Made Bagiarta']
['I Made Bagiarta']
['I Gusti Made Juliartawan']
['I Gusti Made Juliartawan']
['Made Hermayanti Muliartha']
['I Gusti Made Juliartawan']
['I Made Bagiarta']
['I Gusti Ayu Kade Ari Wulandari']
['I Gusti Ayu Kade Ari Wulandari']
['I Made Bagiarta']
['Made Hermayanti Muliartha']
['I Gusti Made Juliartawan']
['I Gusti Made Juliartawan']
['I Made Bagiarta']
['I Made Bagiarta']
['Made Hermayanti Muliartha']
['I Gusti Made Juliartawan']
['I Made Bagiarta']
['I Gusti Made Juliartawan']
['I Made Bagiarta']
['I Made Bagiarta']
['I Gusti Made Juliartawan']
['I Gusti Made Juliartawan']
['I Made Bagiarta']
['Made Hermayanti Muliartha']
['I Gusti Made Juliartawan']
['I Gusti Made Juliartawan']
['Made Hermayanti Muliartha']
['I Made Bagiarta']
['I Gusti Made Juliartawan']
['I Gusti Made Juliartawan']
['I Made Bagiarta']
['Heriyanti']
['I Made Bagiarta']
['Made Hermayanti Muliartha']
['I Gusti Made Juliartawan']
['I Gusti Made Juliartawan']
['I Gusti Made Juliar

In [269]:
df_copy['hakim'] = df_copy['hakim'].apply(lambda x: str(x[0]) if isinstance(x, list) and len(x) > 0 else str(x))

In [270]:
unique_types = df_copy['hakim'].apply(type).unique()
print(unique_types)

[<class 'str'>]


In [271]:
for index, row in df_copy.iterrows():
    print(row['hakim'])

I Made Bagiarta
I Made Bagiarta
I Made Bagiarta
I Gusti Made Juliartawan
I Gusti Made Juliartawan
Made Hermayanti Muliartha
I Gusti Made Juliartawan
I Made Bagiarta
I Gusti Ayu Kade Ari Wulandari
I Gusti Ayu Kade Ari Wulandari
I Made Bagiarta
Made Hermayanti Muliartha
I Gusti Made Juliartawan
I Gusti Made Juliartawan
I Made Bagiarta
I Made Bagiarta
Made Hermayanti Muliartha
I Gusti Made Juliartawan
I Made Bagiarta
I Gusti Made Juliartawan
I Made Bagiarta
I Made Bagiarta
I Gusti Made Juliartawan
I Gusti Made Juliartawan
I Made Bagiarta
Made Hermayanti Muliartha
I Gusti Made Juliartawan
I Gusti Made Juliartawan
Made Hermayanti Muliartha
I Made Bagiarta
I Gusti Made Juliartawan
I Gusti Made Juliartawan
I Made Bagiarta
Heriyanti
I Made Bagiarta
Made Hermayanti Muliartha
I Gusti Made Juliartawan
I Gusti Made Juliartawan
I Gusti Made Juliartawan
Heriyanti
Ni Made Kushandari
Heriyanti
I Gusti Made Juliartawan
I Gusti Made Juliartawan
I Made Bagiarta
I Made Bagiarta
I Gusti Made Juliartawan
I 

### Data Cleaning: Barang Bukti

In [272]:
text = df_copy.iloc[-1]['barang_bukti']
print(text)

M E N G A D I L I
Menyatakan Terdakwa KETUT GELGEL ASTREA identitas seperti tersebut diatas telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana Â â€Pencurianâ€ ;
Menjatuhkan pidana kepada Terdakwa oleh karenanya Â dengan pidana penjara Â selama Â 1 (satu) tahun;
Menetapkan masa penangkapan dan penahanan yang telah dijalani Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan ;
Menetapkan Terdakwa tetap berada dalam tahanan ;
Menetapkan barang bukti berupa ;
1 (satu) unit sepeda motor Yamaha Jupiter Z warna hitam tahun 2006 , Nomor Polisi DK 6540 UN,Noka :MH32P20026K154375, Nosin :2P2-154118;
1 (satu) lembar surat kendaraan STNK atas nama HENDRA;
Dikembalikan kepada saksi korbanÂ  GEDE AGUS DIKA ANDRIANA Â atau yang berhak;
Membebankan kepada Terdakwa untuk membayar biaya perkara sebesar Rp. 5000,- (lima ribu rupiah);Â


In [273]:
cleaned_text = re.sub(r'[^a-zA-Z0-9., /\\()\"\'\n-]+', '', text)

In [473]:
print(df[df['nomor_perkara'] == '86/Pid.B/2019/PN Sgr']['barang_bukti'])


937    M E N G A D I L I :\nMenyatakan Terdakwa NYOMA...
Name: barang_bukti, dtype: object


In [274]:
print(cleaned_text)

M E N G A D I L I
Menyatakan Terdakwa KETUT GELGEL ASTREA identitas seperti tersebut diatas telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana  Pencurian 
Menjatuhkan pidana kepada Terdakwa oleh karenanya  dengan pidana penjara  selama  1 (satu) tahun
Menetapkan masa penangkapan dan penahanan yang telah dijalani Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan 
Menetapkan Terdakwa tetap berada dalam tahanan 
Menetapkan barang bukti berupa 
1 (satu) unit sepeda motor Yamaha Jupiter Z warna hitam tahun 2006 , Nomor Polisi DK 6540 UN,Noka MH32P20026K154375, Nosin 2P2-154118
1 (satu) lembar surat kendaraan STNK atas nama HENDRA
Dikembalikan kepada saksi korban  GEDE AGUS DIKA ANDRIANA  atau yang berhak
Membebankan kepada Terdakwa untuk membayar biaya perkara sebesar Rp. 5000,- (lima ribu rupiah)


In [275]:
cleaned_text = re.sub(r' +', ' ', cleaned_text)

In [276]:
print(cleaned_text)

M E N G A D I L I
Menyatakan Terdakwa KETUT GELGEL ASTREA identitas seperti tersebut diatas telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana Pencurian 
Menjatuhkan pidana kepada Terdakwa oleh karenanya dengan pidana penjara selama 1 (satu) tahun
Menetapkan masa penangkapan dan penahanan yang telah dijalani Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan 
Menetapkan Terdakwa tetap berada dalam tahanan 
Menetapkan barang bukti berupa 
1 (satu) unit sepeda motor Yamaha Jupiter Z warna hitam tahun 2006 , Nomor Polisi DK 6540 UN,Noka MH32P20026K154375, Nosin 2P2-154118
1 (satu) lembar surat kendaraan STNK atas nama HENDRA
Dikembalikan kepada saksi korban GEDE AGUS DIKA ANDRIANA atau yang berhak
Membebankan kepada Terdakwa untuk membayar biaya perkara sebesar Rp. 5000,- (lima ribu rupiah)


In [277]:
pattern = r"Menetapkan barang bukti berupa\s*(.*?)(?=\n\s*[A-Z]|$)"
# pattern = r"Menetapkan barang bukti berupa\s*(.*?)(?=\n[A-Z])"
match = re.search(pattern, cleaned_text, re.DOTALL)

if match:
    print(match.group().strip())
else:
    print("Tidak ada kecocokan ditemukan.")

Menetapkan barang bukti berupa 
1 (satu) unit sepeda motor Yamaha Jupiter Z warna hitam tahun 2006 , Nomor Polisi DK 6540 UN,Noka MH32P20026K154375, Nosin 2P2-154118
1 (satu) lembar surat kendaraan STNK atas nama HENDRA


In [278]:
text = df_copy.iloc[3]['barang_bukti']
print(text)

MENGADILI:
Menyatakan Terdakwa SANG PUTU WIDIANA telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana “dengan permufakatan jahat melakukan tindak pidana narkotika dan precursor narkotika yaitu tanpa hak atau melawan hukum memiliki, menyimpan, menguasai atau menyediakan narkotika Golongan I bukan tanaman” sebagaimana  dakwaan alternatif kedua Penuntut Umum;
Menjatuhkan pidana terhadap Terdakwa berupa pidana penjara selama 4 (empat) tahun dan denda sebesar Rp800.000.000,00 (delapan ratus juta rupiah) dengan ketentuan apabila denda tersebut tidak dibayarkan diganti dengan pidana penjara selama 2 (dua) bulan;      
Menetapkan masa penangkapan dan penahanan yang telah dijalani oleh Terdakwa dikurangkan seluruhnya dari pidana penjara yang dijatuhkan;
Menetapkan Terdakwa tetap ditahan ;
Menetapkan barang bukti berupa:
1(satu) buah HP merk Realme warna hitam putih
Dirampas untuk dimusnahkan
6 (enam) Paket plastik klip bening yang berisi butiran kristal bening diduga narkot

In [279]:
patterns = [
    r"barang\s*bukti\s*berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"Barang\s*bukti\s*berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"Barang\s*Bukti\s*berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"Barang\s*Bukti\s*Berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"barang\s*Bukti\s*Berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"barang\s*bukti\s*Berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"barang\s*bukti,\s*berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"barang\s*bukti\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
    r"barang-bukti\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)"
]

# List to store indices of rows to drop
rows_to_drop = []

# Iterate over rows
for index, row in df_copy.iterrows():
    text_barang_bukti = row['barang_bukti']
    
    # Clean the 'barang_bukti' text
    cleaned_barang_bukti = re.sub(r'[^a-zA-Z0-9., /\\()\"\'\n-]+', '', text_barang_bukti)
    
    match_found = False

    # Iterate over the patterns and try to find a match
    for pattern in patterns:
        match = re.search(pattern, cleaned_barang_bukti, re.DOTALL)

        if match:
            df_copy.loc[index, 'cleaned_barang_bukti'] = match.group(1).strip().replace('\n', ' ')  # Store the match
            match_found = True
            break

    
    if not match_found:
        # Print info for unmatched rows
        print(f"{index}. Tidak ada kecocokan ditemukan. {row['nomor_perkara']}")
        rows_to_drop.append(index)  # Collect row index to drop
        print(cleaned_barang_bukti)

# Drop rows after the loop completes
df_copy.drop(rows_to_drop, inplace=True)
print(f"rows dropped: {rows_to_drop}")
print(f"rows dropped: {len(rows_to_drop)}")

36. Tidak ada kecocokan ditemukan. 68/Pid.B/2024/PN Sgr
MENGADILI
Menyatakan Terdakwa I Nyoman Eryanto Alias Munuk telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana penganiayaan sebagaimana dalam dakwaan tunggal
Menjatuhkan pidana terhadap Terdakwa dengan pidana penjara selama 4  (empat) bulan       
Menetapkan masa penangkapan dan penahanan yang telah dijalani Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan
Menetapkan agar Terdakwa tetap ditahan
Membebankan kepada Terdakwa untuk membayar biaya perkara sebesar Rp. 5.000. (lima ribu rupiah)
100. Tidak ada kecocokan ditemukan. 149/Pid.B/2023/PN Sgr
MENGADILI
Menyatakan Terdakwa PUTU HERRY SUARTHANA Alias HERIK sebagaimana identitas tersebut di atas, telah terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana penganiayaan sebagaimana dalam dakwaan tunggal  Penuntut Umum 
Menjatuhkan pidana terhadap Terdakwa oleh karena itu dengan pidana penjara selama 1 ( satu ) tahun
Menetapkan lamanya ma

In [280]:
df_copy.iloc[0]['cleaned_barang_bukti']

'1 (satu) buah papan bola bergambar 1 (satu) buah tas kain warna abu-abu 1 (satu) buah perlak bergambar 1 (satu) buah tas karung warna biru garis merah 2 (dua)  buah bola karet warna hitam 1 (satu) buah bola karet warna cokelat 1 (satu) buah bola karet warna hijau 1 (satu) buah tas kain warna hitam 4 (empat) buah kayu penyeimbang 1 (satu) botol bedak bayi 48 (empat puluh delapan) buah papan cek kayu berwarna hijau dengan nomor 5 (lima) 68 (enam puluh delapan) buah papan cek kayu berwarna merah dengan nomor 10 (sepuluh) 44 (empat puluh empat) buah papan cek kayu berwarna biru dengan nomor 20 (duapuluh) 17 (tujuh belas) buah papan cek kayu berwarna abu-abu dengan nomor 30 (tigapuluh) 17 (tujuh belas) buah papan cek kayu berwarna ungu dengan nomor 50 (limapuluh) 1 (satu) buah lap warna orange 1 (satu) buah lap warna putih kotak warna biru'

In [474]:
print(df_copy[df_copy['nomor_perkara'] == '86/Pid.B/2019/PN Sgr']['cleaned_barang_bukti'])


803    
Name: cleaned_barang_bukti, dtype: object


In [281]:
df_copy

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti
0,Minutasi,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"Kamis, 08 Agu. 2024",41 Hari,"['KETUT SUBAGIA', 'KOMANG ADITYA']","['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,3,['KOMANG ADITYA~Pidana Penjara Waktu Tertentu ...,MENGADILI:\nMenyatakan Terdakwa I. Ketut Subag...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah papan bola bergambar 1 (satu) bu...
1,Minutasi,118/Pid.Sus/2024/PN Sgr,Narkotika,"Senin, 05 Agu. 2024",44 Hari,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,MENGADILI:\nMenyatakan Terdakwa I NYOMAN SUART...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,38 (tiga puluh delapan) potongan pipet plastik...
2,Minutasi,114/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",48 Hari,['RICO JAYADI'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['RICO JAYADI~Subsider Penjara (3 Bulan )\nPid...,MENGADILI:\nMenyatakan Terdakwa Rico Jayadi te...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah dompet warna hijau 20 (dua puluh...
3,Minutasi,115/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",53 Hari,['SANG PUTU WIDIANA'],"['Isnarti Jayaningsih, S.H.']",I Gusti Made Juliartawan,4,['SANG PUTU WIDIANA~Subsider Penjara (2 Bulan ...,MENGADILI:\nMenyatakan Terdakwa SANG PUTU WIDI...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1(satu) buah HP merk Realme warna hitam putih
4,Minutasi,116/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",53 Hari,['WAYAN DARNA MASTONO'],"['Isnarti Jayaningsih, S.H.']",I Gusti Made Juliartawan,4,['WAYAN DARNA MASTONO~Subsider Penjara (2 Bula...,MENGADILI:\nMenyatakan Terdakwa WAYAN DARNA MA...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,6 (enam) Paket plastik klip bening yang berisi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,Minutasi,223/Pid.Sus/2017/PN Sgr,Perlindungan dan Pengelolaan Lingkungan Hidup,"Kamis, 07 Des. 2017",60 Hari,[Jumat Ariyanto],"[I Nyoman Sulitra, SH.,MH.]",Ida Bagus Bama Dewa. P,3,[Jumat Ariyanto~Pidana Kurungan (1 Tahun )\nSu...,M E N G A D I L I\nÂ \nMenyatakan Terdakwa Jum...,Kesatu : Pasal 83 ayat (1) huruf b UU RI No. 1...,1 (satu) 1 (satu) batang kayu balang-balang be...
1087,Minutasi,224/Pid.B/2017/PN Sgr,Kejahatan Perjudian,"Kamis, 07 Des. 2017",48 Hari,[I KETUT ANA SAPUTRA Alias BENCUT],"[MADE ASTINI, SH.]",Sudar,1,[I KETUT ANA SAPUTRA Alias BENCUT~Pidana Penja...,Menyatakan Terdakwa I Ketut Ana Saputra Alias ...,Perbuatan terdakwa sebagaimana diatur dan dian...,36 (tiga puluh enam) lembar kertas rekapan
1088,Minutasi,220/Pid.B/2017/PN Sgr,Penganiayaan,"Rabu, 06 Des. 2017",57 Hari,[Sahabudin Alias Udin],"[I KETUT KINDRA, SH]",Ni Luh Suantini,2,[Sahabudin Alias Udin~Pidana Penjara Waktu Ter...,M E N G A D I L I :\nÂ \n1. Menyatakan terdakw...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...,1. 1 (satu) buah tabung gas elpiji ukuran 3 (t...
1089,Minutasi,219/Pid.B/2017/PN Sgr,Kejahatan yang Membahayakan Keamananan Umum Ba...,"Rabu, 06 Des. 2017",91 Hari,[Putu Kristian Damanta],"[I GUSTI ARYA WIDNYANA, SH.]",A.A. Sagung Yuni Wulantrisna,4,[Putu Kristian Damanta~Pidana Penjara Waktu Te...,"1. Menyatakan Terdakwa Putu Kristian Damanta ,...",Pertama : Pasal 187 ayat (1) KUHP\ndan\nKedua ...,-1 (satu)unit sepeda motor merk Yamaha Jupiter...


In [282]:
null_counts = df_copy.isnull().sum()

print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara          0
nomor_perkara           0
klasifikasi_perkara     0
tanggal_pendaftaran     0
lama_proses             0
terdakwa                0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
putusan_hukuman         0
barang_bukti            0
dakwaan                 0
cleaned_barang_bukti    0
dtype: int64


### Data Cleaning: Dakwaan

In [283]:
dakwaan = df_copy.iloc[1]['dakwaan']
print(dakwaan)

KEJAKSAAN REPUBLIK INDONESIA
KEJAKSAAN TINGGI BALI
KEJAKSAAN NEGERI BULELENG
JL. Dewi Sartika Selatan No. 23 Singaraja – Bali 81116
Telp. (0362) 22580 www.kejari-buleleng.go.id
"Demi Keadilan dan Kebenaran
Berdasarkan Ketuhanan Yang Maha Esa"
P-29
        SURAT DAKWAAN
NOMOR REG PERK : PDM –  44 /Enz.2/BLL/07/2024
  A.     IDENTITAS TERDAKWA  :
Nama Lengkap             :    I NYOMAN SUARTA
Nomor Identitas             :    5103051109720003
Tempat lahir                  :    Wanagiri
Umur/Tgl lahir               :    51 Tahun / 11 September 1972
Jenis Kelamin                :    Laki-laki      
Kebangsaan                  :    Indonesia
Tempat tinggal               :    Banjar Dinas Yeh Ketipat, Desa Wanagiri, Kecamatan Sukasada, Kabupaten Buleleng
Agama                           :    Hindu
Pekerjaan                      :    Wiraswasta
Pendidikan                    :    S1
  B.     STATUS PENANGKAPAN DAN PENAHANAN :   
1      Penangkapan                
Penangkapan                      

In [284]:
count_short_texts = sum(1 for text in df_copy['dakwaan'] if len(text) < 500)
print(f"Jumlah teks dengan panjang di bawah 500 karakter: {count_short_texts}")

Jumlah teks dengan panjang di bawah 500 karakter: 15


In [285]:
# cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n]+', dakwaan)

# cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)

# print(cleaned_dakwaan)

cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n-]+', dakwaan)

cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)
cleaned_dakwaan = re.sub(r'-{2,}', '-', cleaned_dakwaan)

print(cleaned_dakwaan)


KEJAKSAAN REPUBLIK INDONESIA
KEJAKSAAN TINGGI BALI
KEJAKSAAN NEGERI BULELENG
JL. Dewi Sartika Selatan No. 23 Singaraja  Bali 81116
Telp. (0362) 22580 www.kejari-buleleng.go.id
"Demi Keadilan dan Kebenaran
Berdasarkan Ketuhanan Yang Maha Esa"
P-29
        SURAT DAKWAAN
NOMOR REG PERK  PDM   44 /Enz.2/BLL/07/2024
  A.     IDENTITAS TERDAKWA  
Nama Lengkap                 I NYOMAN SUARTA
Nomor Identitas                 5103051109720003
Tempat lahir                      Wanagiri
Umur/Tgl lahir                   51 Tahun / 11 September 1972
Jenis Kelamin                    Laki-laki      
Kebangsaan                      Indonesia
Tempat tinggal                   Banjar Dinas Yeh Ketipat, Desa Wanagiri, Kecamatan Sukasada, Kabupaten Buleleng
Agama                               Hindu
Pekerjaan                          Wiraswasta
Pendidikan                        S1
  B.     STATUS PENANGKAPAN DAN PENAHANAN    
1      Penangkapan                
Penangkapan                              Tanggal

In [286]:
def clean_dakwaan(text):
    # Extract unwanted characters
    cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n-]+', text)
    cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)
    
    # Replace multiple dashes with a single dash
    cleaned_dakwaan = re.sub(r'-{2,}', '-', cleaned_dakwaan)
    
    return cleaned_dakwaan

In [287]:
# Apply the cleaning function row-wise
df_copy['cleaned_dakwaan'] = df_copy['dakwaan'].apply(clean_dakwaan)

In [288]:
df_copy

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti,cleaned_dakwaan
0,Minutasi,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"Kamis, 08 Agu. 2024",41 Hari,"['KETUT SUBAGIA', 'KOMANG ADITYA']","['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,3,['KOMANG ADITYA~Pidana Penjara Waktu Tertentu ...,MENGADILI:\nMenyatakan Terdakwa I. Ketut Subag...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah papan bola bergambar 1 (satu) bu...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
1,Minutasi,118/Pid.Sus/2024/PN Sgr,Narkotika,"Senin, 05 Agu. 2024",44 Hari,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,MENGADILI:\nMenyatakan Terdakwa I NYOMAN SUART...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,38 (tiga puluh delapan) potongan pipet plastik...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
2,Minutasi,114/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",48 Hari,['RICO JAYADI'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['RICO JAYADI~Subsider Penjara (3 Bulan )\nPid...,MENGADILI:\nMenyatakan Terdakwa Rico Jayadi te...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah dompet warna hijau 20 (dua puluh...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
3,Minutasi,115/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",53 Hari,['SANG PUTU WIDIANA'],"['Isnarti Jayaningsih, S.H.']",I Gusti Made Juliartawan,4,['SANG PUTU WIDIANA~Subsider Penjara (2 Bulan ...,MENGADILI:\nMenyatakan Terdakwa SANG PUTU WIDI...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1(satu) buah HP merk Realme warna hitam putih,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
4,Minutasi,116/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",53 Hari,['WAYAN DARNA MASTONO'],"['Isnarti Jayaningsih, S.H.']",I Gusti Made Juliartawan,4,['WAYAN DARNA MASTONO~Subsider Penjara (2 Bula...,MENGADILI:\nMenyatakan Terdakwa WAYAN DARNA MA...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,6 (enam) Paket plastik klip bening yang berisi...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,Minutasi,223/Pid.Sus/2017/PN Sgr,Perlindungan dan Pengelolaan Lingkungan Hidup,"Kamis, 07 Des. 2017",60 Hari,[Jumat Ariyanto],"[I Nyoman Sulitra, SH.,MH.]",Ida Bagus Bama Dewa. P,3,[Jumat Ariyanto~Pidana Kurungan (1 Tahun )\nSu...,M E N G A D I L I\nÂ \nMenyatakan Terdakwa Jum...,Kesatu : Pasal 83 ayat (1) huruf b UU RI No. 1...,1 (satu) 1 (satu) batang kayu balang-balang be...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...
1087,Minutasi,224/Pid.B/2017/PN Sgr,Kejahatan Perjudian,"Kamis, 07 Des. 2017",48 Hari,[I KETUT ANA SAPUTRA Alias BENCUT],"[MADE ASTINI, SH.]",Sudar,1,[I KETUT ANA SAPUTRA Alias BENCUT~Pidana Penja...,Menyatakan Terdakwa I Ketut Ana Saputra Alias ...,Perbuatan terdakwa sebagaimana diatur dan dian...,36 (tiga puluh enam) lembar kertas rekapan,Perbuatan terdakwa sebagaimana diatur dan dian...
1088,Minutasi,220/Pid.B/2017/PN Sgr,Penganiayaan,"Rabu, 06 Des. 2017",57 Hari,[Sahabudin Alias Udin],"[I KETUT KINDRA, SH]",Ni Luh Suantini,2,[Sahabudin Alias Udin~Pidana Penjara Waktu Ter...,M E N G A D I L I :\nÂ \n1. Menyatakan terdakw...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...,1. 1 (satu) buah tabung gas elpiji ukuran 3 (t...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...
1089,Minutasi,219/Pid.B/2017/PN Sgr,Kejahatan yang Membahayakan Keamananan Umum Ba...,"Rabu, 06 Des. 2017",91 Hari,[Putu Kristian Damanta],"[I GUSTI ARYA WIDNYANA, SH.]",A.A. Sagung Yuni Wulantrisna,4,[Putu Kristian Damanta~Pidana Penjara Waktu Te...,"1. Menyatakan Terdakwa Putu Kristian Damanta ,...",Pertama : Pasal 187 ayat (1) KUHP\ndan\nKedua ...,-1 (satu)unit sepeda motor merk Yamaha Jupiter...,Pertama Pasal 187 ayat (1) KUHP\ndan\nKedua ...


In [289]:
print(df_copy.iloc[0]["cleaned_dakwaan"])

KEJAKSAAN REPUBLIK INDONESIA
KEJAKSAAN TINGGI BALI
KEJAKSAAN  NEGERI  BULELENG
Jl. Dewi Sartika No. 23, Kaliuntu, Kec. Buleleng, Kab. Buleleng, Bali
Telp. (0362) 22580. www.kejari-buleleng.go.id.
"Demi Keadilan dan Kebenaran Berdasarkan Ketuhanan Yang Maha Esa"
P-29
          SURAT DAKWAAN
NOMOR REG. PERKARA  PDM- 19/Eku.2/Bll/07/2024.
  IDENTITAS TERDAKWA 
Nama Lengkap

KETUT SUBAGIA
Nomor Identitas

5108010205860005
Tempat lahir

Gondol
Umur/tanggal lahir

38 Tahun/ 02 Mei 1986
Jenis Kelamin

Laki-laki
Kebangsaan/Kewarganegaraan

Indonesia
Tempat Tinggal

Banjar Dinas Gondol, Desa Penyabangan, Kecamatan Gerokgak, Kabupaten Buleleng
A g a m a

Hindu
Pekerjaan

Karyawan Swasta
Pendidikan

SD
      Nama Lengkap

KOMANG ADITYA
Nomor Identitas

5108010107020184
Tempat lahir

Penyabangan
Umur/tanggal lahir

22  Tahun/ 01 November 2001
Jenis Kelamin

Laki-laki
Kebangsaan/Kewarganegaraan

Indonesia
Tempat Tinggal

Banjar Dinas Triamerta, RT/RW-/-, Desa Penyabangan, Kecamatan Gerokgak, Kabupa

In [290]:
pattern = r"(?i)dakwaan(?!\nnomor)[\s\S]*?(?=(?:dakwaan|$))"

for index, row in df_copy.iterrows():
    text_dakwaan = row['cleaned_dakwaan']
    matches = re.findall(pattern, text_dakwaan, re.IGNORECASE)

    if matches: 
        last_match = matches[-1]
        df_copy.loc[index, "cleaned_dakwaan"] = last_match.strip()
    else:
        df_copy.loc[index, "cleaned_dakwaan"] = text_dakwaan
        print(f"raw dakwaan copied at {index}")


raw dakwaan copied at 7
raw dakwaan copied at 39
raw dakwaan copied at 74
raw dakwaan copied at 89
raw dakwaan copied at 90
raw dakwaan copied at 91
raw dakwaan copied at 92
raw dakwaan copied at 94
raw dakwaan copied at 95
raw dakwaan copied at 96
raw dakwaan copied at 97
raw dakwaan copied at 98
raw dakwaan copied at 99
raw dakwaan copied at 102
raw dakwaan copied at 103
raw dakwaan copied at 105
raw dakwaan copied at 106
raw dakwaan copied at 107
raw dakwaan copied at 108
raw dakwaan copied at 110
raw dakwaan copied at 111
raw dakwaan copied at 112
raw dakwaan copied at 113
raw dakwaan copied at 115
raw dakwaan copied at 116
raw dakwaan copied at 118
raw dakwaan copied at 119
raw dakwaan copied at 120
raw dakwaan copied at 121
raw dakwaan copied at 124
raw dakwaan copied at 125
raw dakwaan copied at 126
raw dakwaan copied at 127
raw dakwaan copied at 128
raw dakwaan copied at 132
raw dakwaan copied at 133
raw dakwaan copied at 134
raw dakwaan copied at 135
raw dakwaan copied at 136


In [291]:
df_copy

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti,cleaned_dakwaan
0,Minutasi,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"Kamis, 08 Agu. 2024",41 Hari,"['KETUT SUBAGIA', 'KOMANG ADITYA']","['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,3,['KOMANG ADITYA~Pidana Penjara Waktu Tertentu ...,MENGADILI:\nMenyatakan Terdakwa I. Ketut Subag...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN\n PERTAMA \n- Bahwa Terdakwa KETUT SU...
1,Minutasi,118/Pid.Sus/2024/PN Sgr,Narkotika,"Senin, 05 Agu. 2024",44 Hari,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,MENGADILI:\nMenyatakan Terdakwa I NYOMAN SUART...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN \nKESATU\n Bahwa ia Terdak...
2,Minutasi,114/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",48 Hari,['RICO JAYADI'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['RICO JAYADI~Subsider Penjara (3 Bulan )\nPid...,MENGADILI:\nMenyatakan Terdakwa Rico Jayadi te...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah dompet warna hijau 20 (dua puluh...,DAKWAAN \nKESATU\n ...
3,Minutasi,115/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",53 Hari,['SANG PUTU WIDIANA'],"['Isnarti Jayaningsih, S.H.']",I Gusti Made Juliartawan,4,['SANG PUTU WIDIANA~Subsider Penjara (2 Bulan ...,MENGADILI:\nMenyatakan Terdakwa SANG PUTU WIDI...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1(satu) buah HP merk Realme warna hitam putih,Dakwaan\nKe Satu\n- Bahwa terdakwa SANG PUTU...
4,Minutasi,116/Pid.Sus/2024/PN Sgr,Narkotika,"Kamis, 01 Agu. 2024",53 Hari,['WAYAN DARNA MASTONO'],"['Isnarti Jayaningsih, S.H.']",I Gusti Made Juliartawan,4,['WAYAN DARNA MASTONO~Subsider Penjara (2 Bula...,MENGADILI:\nMenyatakan Terdakwa WAYAN DARNA MA...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,6 (enam) Paket plastik klip bening yang berisi...,dakwaan petugas Sat Narkoba Polres Buleleng m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,Minutasi,223/Pid.Sus/2017/PN Sgr,Perlindungan dan Pengelolaan Lingkungan Hidup,"Kamis, 07 Des. 2017",60 Hari,[Jumat Ariyanto],"[I Nyoman Sulitra, SH.,MH.]",Ida Bagus Bama Dewa. P,3,[Jumat Ariyanto~Pidana Kurungan (1 Tahun )\nSu...,M E N G A D I L I\nÂ \nMenyatakan Terdakwa Jum...,Kesatu : Pasal 83 ayat (1) huruf b UU RI No. 1...,1 (satu) 1 (satu) batang kayu balang-balang be...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...
1087,Minutasi,224/Pid.B/2017/PN Sgr,Kejahatan Perjudian,"Kamis, 07 Des. 2017",48 Hari,[I KETUT ANA SAPUTRA Alias BENCUT],"[MADE ASTINI, SH.]",Sudar,1,[I KETUT ANA SAPUTRA Alias BENCUT~Pidana Penja...,Menyatakan Terdakwa I Ketut Ana Saputra Alias ...,Perbuatan terdakwa sebagaimana diatur dan dian...,36 (tiga puluh enam) lembar kertas rekapan,Perbuatan terdakwa sebagaimana diatur dan dian...
1088,Minutasi,220/Pid.B/2017/PN Sgr,Penganiayaan,"Rabu, 06 Des. 2017",57 Hari,[Sahabudin Alias Udin],"[I KETUT KINDRA, SH]",Ni Luh Suantini,2,[Sahabudin Alias Udin~Pidana Penjara Waktu Ter...,M E N G A D I L I :\nÂ \n1. Menyatakan terdakw...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...,1. 1 (satu) buah tabung gas elpiji ukuran 3 (t...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...
1089,Minutasi,219/Pid.B/2017/PN Sgr,Kejahatan yang Membahayakan Keamananan Umum Ba...,"Rabu, 06 Des. 2017",91 Hari,[Putu Kristian Damanta],"[I GUSTI ARYA WIDNYANA, SH.]",A.A. Sagung Yuni Wulantrisna,4,[Putu Kristian Damanta~Pidana Penjara Waktu Te...,"1. Menyatakan Terdakwa Putu Kristian Damanta ,...",Pertama : Pasal 187 ayat (1) KUHP\ndan\nKedua ...,-1 (satu)unit sepeda motor merk Yamaha Jupiter...,Pertama Pasal 187 ayat (1) KUHP\ndan\nKedua ...


In [292]:
print(df_copy.iloc[132]['cleaned_dakwaan'])

-Bahwa ia Terdakwa WAYAN GATRA YASA ALIAS WAYAN POM SENDANG secara bersama  sama dengan saksi Mas Arif Efendi (terdakwa dalam berkas perkara terpisah) pada hari Minggu, tanggal 12 Maret 2023 sekira pukul 15.00 WITA atau setidak-tidaknya pada suatu waktu dalam bulan Maret di tahun 2023, bertempat di Stasiun Pengisian Bahan Bakar Umum (SPBU) 54.811.12 Pemuteran-Singaraja yang beralamat di Jalan Seririt Gilimanuk, Desa Pemuteran, Kec. Gerokgak, Kab. Buleleng, Prov. Bali atau setidak-tidaknya pada tempat tertentu dimana Pengadilan Negeri Singaraja  berwenang memeriksa dan mengadili, baik mereka yang melakukan, yang menyuruh melakukan dan yang turut serta melakukan perbuatan, menyalahgunakan Pengangkutan dan/atau Niaga Bahan Bakar Minyak, bahan bakar gas,dan /atau liquefied petroleum gas yang disubsidi dan/atau penyediaan dan pendistribusiannya diberikan penugasan Pemerintah, yang dilakukan oleh terdakwa dengan cara- cara  sebagai berikut 
Bahwa berawal pada hari Minggu tanggal 12 Maret 202

In [293]:

pattern = r"^(.*?)\s+Singaraja,\s+\d{1,2}\s+\w+\s+\d{4}"

for index, row in df_copy.iterrows():
    text_dakwaan = row['cleaned_dakwaan']
    matches = re.match(pattern, text_dakwaan, re.DOTALL)

    if matches: 
        df_copy.loc[index, "cleaned_dakwaan"] = matches.group(1).strip()
    else:
        df_copy.loc[index, "cleaned_dakwaan"] = text_dakwaan
        print(f"raw dakwaan copied at {index}")


raw dakwaan copied at 7
raw dakwaan copied at 33
raw dakwaan copied at 39
raw dakwaan copied at 69
raw dakwaan copied at 85
raw dakwaan copied at 86
raw dakwaan copied at 87
raw dakwaan copied at 88
raw dakwaan copied at 89
raw dakwaan copied at 90
raw dakwaan copied at 91
raw dakwaan copied at 92
raw dakwaan copied at 94
raw dakwaan copied at 95
raw dakwaan copied at 96
raw dakwaan copied at 97
raw dakwaan copied at 98
raw dakwaan copied at 99
raw dakwaan copied at 102
raw dakwaan copied at 103
raw dakwaan copied at 104
raw dakwaan copied at 105
raw dakwaan copied at 106
raw dakwaan copied at 107
raw dakwaan copied at 108
raw dakwaan copied at 110
raw dakwaan copied at 111
raw dakwaan copied at 112
raw dakwaan copied at 113
raw dakwaan copied at 114
raw dakwaan copied at 115
raw dakwaan copied at 116
raw dakwaan copied at 117
raw dakwaan copied at 118
raw dakwaan copied at 119
raw dakwaan copied at 120
raw dakwaan copied at 121
raw dakwaan copied at 124
raw dakwaan copied at 125
raw d

In [294]:
print(df_copy.iloc[108]['cleaned_dakwaan'])

Kesatu
-Bahwa Terdakwa I NYOMAN MAHARDIKA Als. MAHA  pada hari Jumat  tanggal 25 Agustus  2023 sekira pukul 20.30 Wita atau setidak-tidaknya pada suatu waktu dalam bulan Agustus 2023 atau setidak-tidaknya pada suatu waktu dalam Tahun 2023, bertempat di Traffic Light Lovina Jalan Singaraja  Seririt Desa Kalibukbuk Kecamatan Buleleng Kabupaten Buleleng atau setidaktidaknya pada suatu tempat yang masih termasuk dalam daerah hukum Pengadilan Negeri Singaraja yang berwenang memeriksa dan mengadili perkara ini, Tanpa hak atau melawan hukum menawarkan untuk dijual, menjual, membeli, menerima, menjadi perantara dalam jual beli, menukar atau menyerahkan narkotika golongan I jenis Shabu
Perbuatan terdakwa sebagaimana diatur dan diancam pidana dalam Pasal 114 ayat (1) Undang- Undang Republik Indonesia  Nomor 35 tahun 2009 tentang Narkotika
atau
KEDUA
- Bahwa Terdakwa I NYOMAN MAHARDIKA Als. MAHA  pada hari Jumat  tanggal 25 Agustus  2023 sekira pukul 20.30 Wita atau setidak-tidaknya pada suatu wa

In [295]:
df_copy.to_csv(f'../Data/STAGE 1 PREPROCESSING/STAGE_1_PREPROCESSING_{database_name}.csv', index=False)

## Data Transformation

In [357]:
df_clean = pd.read_csv(f'../Data/STAGE 1 PREPROCESSING/STAGE_1_PREPROCESSING_{database_name}.csv')

In [471]:
# Print the 'cleaned_barang_bukti' column for rows where 'nomor_perkara' is '86/Pid.B/2019/PN Sgr'
print(df_copy[df_copy['nomor_perkara'] == '86/Pid.B/2019/PN Sgr']['cleaned_barang_bukti'])


803    
Name: cleaned_barang_bukti, dtype: object


### Data Transformation: Copy to New DataFrame

In [358]:
df_clean.head(1)

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti,cleaned_dakwaan
0,Minutasi,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"Kamis, 08 Agu. 2024",41 Hari,"['KETUT SUBAGIA', 'KOMANG ADITYA']","['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,3,['KOMANG ADITYA~Pidana Penjara Waktu Tertentu ...,MENGADILI:\nMenyatakan Terdakwa I. Ketut Subag...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN\n PERTAMA \n- Bahwa Terdakwa KETUT SU...


In [359]:
selected_columns = ['nomor_perkara', 'klasifikasi_perkara', 'terdakwa', 'penuntut_umum', 'hakim', 'jumlah_saksi', 'putusan_hukuman', 'cleaned_barang_bukti', 'cleaned_dakwaan']
df_clean = df_clean[selected_columns].copy()

df_clean.head(1)

,nomor_perkara,klasifikasi_perkara,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,cleaned_barang_bukti,cleaned_dakwaan
0,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"['KETUT SUBAGIA', 'KOMANG ADITYA']","['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,3,['KOMANG ADITYA~Pidana Penjara Waktu Tertentu ...,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN\n PERTAMA \n- Bahwa Terdakwa KETUT SU...


### Data Transformation: Extract Multiple Terdakwa & Putusan Hukuman to each Row 

In [360]:
print(f"Jumlah Data: {len(df_clean)}")

count_str_terdakwa = df_clean[df_clean["terdakwa"].apply(lambda x: isinstance(x, str))].shape[0]
print(f"Jumlah data dengan 'terdakwa' berupa string: {count_str_terdakwa}")

count_str_putusan_hukuman = df_clean[df_clean["putusan_hukuman"].apply(lambda x: isinstance(x, str))].shape[0]
print(f"Jumlah data dengan 'putusan_hukuman' berupa string: {count_str_terdakwa}")

Jumlah Data: 1031
Jumlah data dengan 'terdakwa' berupa string: 1031
Jumlah data dengan 'putusan_hukuman' berupa string: 1031


In [361]:
def clean_and_convert_to_list(terdakwa):
    # Ensure the input is treated as a string
    if isinstance(terdakwa, list):
        # If it's already a list, return it as is
        return terdakwa
    if isinstance(terdakwa, str):
        # Clean and convert only if it's a string
        cleaned_str = re.sub(r'\s+', ' ', terdakwa.strip())  # Clean whitespace
        try:
            # Use literal_eval to convert if it's a proper list
            result_list = ast.literal_eval(cleaned_str)
        except (ValueError, SyntaxError):
            # If there's an error, just return as a single-element list
            result_list = [cleaned_str]

        # Ensure the result is a list
        if isinstance(result_list, str):
            return [result_list]
        return result_list
    return []


# Apply the cleaning and conversion function
df_clean['transformed_terdakwa'] = df_clean['terdakwa'].apply(clean_and_convert_to_list)
df_clean['transformed_putusan_hukuman'] = df_clean['putusan_hukuman'].apply(clean_and_convert_to_list)

In [362]:
print(df_clean['transformed_terdakwa'].apply(type).unique())
print(df_clean['transformed_putusan_hukuman'].apply(type).unique())
print(f"Jumlah Data: {len(df_clean)}")

# print(type(df_clean.iloc[1]['terdakwa']))
count_str_terdakwa = df_clean[df_clean["transformed_terdakwa"].apply(lambda x: isinstance(x, list))].shape[0]
print(f"Jumlah data dengan 'transformed_terdakwa' berupa list: {count_str_terdakwa}")

count_str_putusan_hukuman = df_clean[df_clean["transformed_putusan_hukuman"].apply(lambda x: isinstance(x, list))].shape[0]
print(f"Jumlah data dengan 'transformed_putusan_hukuman' berupa list: {count_str_terdakwa}")


[<class 'list'>]
[<class 'list'>]
Jumlah Data: 1031
Jumlah data dengan 'transformed_terdakwa' berupa list: 1031
Jumlah data dengan 'transformed_putusan_hukuman' berupa list: 1031


In [363]:
df_clean['transformed_terdakwa'].apply(len).unique()

array([2, 1, 4, 3, 5])

In [364]:
print(df_clean.loc[18, "putusan_hukuman"])
print(df_clean.loc[18, "terdakwa"])


['MOCH HASAN BASRI alias BAS~Pidana Denda Rp.5.000.000,00\nPidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )\nSubsider Kurungan (3 Bulan )', 'KETUT SUMANTRA alias LOTOT~Subsider Kurungan (3 Bulan )\nPidana Denda Rp.5.000.000,00\nPidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )', 'MOCH HASAN BASRI alias BAS~Pidana Denda Rp.5.000.000,00\nPidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )\nSubsider Kurungan (3 Bulan )']
['KETUT SUMANTRA alias LOTOT', 'MOCH HASAN BASRI alias BAS']


In [365]:
df_clean

,nomor_perkara,klasifikasi_perkara,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,cleaned_barang_bukti,cleaned_dakwaan,transformed_terdakwa,transformed_putusan_hukuman
0,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"['KETUT SUBAGIA', 'KOMANG ADITYA']","['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,3,['KOMANG ADITYA~Pidana Penjara Waktu Tertentu ...,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN\n PERTAMA \n- Bahwa Terdakwa KETUT SU...,"[KETUT SUBAGIA, KOMANG ADITYA]",[KOMANG ADITYA~Pidana Penjara Waktu Tertentu (...
1,118/Pid.Sus/2024/PN Sgr,Narkotika,['I NYOMAN SUARTA'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\...,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN \nKESATU\n Bahwa ia Terdak...,[I NYOMAN SUARTA],[I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\n...
2,114/Pid.Sus/2024/PN Sgr,Narkotika,['RICO JAYADI'],"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,['RICO JAYADI~Subsider Penjara (3 Bulan )\nPid...,1 (satu) buah dompet warna hijau 20 (dua puluh...,DAKWAAN \nKESATU\n ...,[RICO JAYADI],[RICO JAYADI~Subsider Penjara (3 Bulan )\nPida...
3,115/Pid.Sus/2024/PN Sgr,Narkotika,['SANG PUTU WIDIANA'],"['Isnarti Jayaningsih, S.H.']",I Gusti Made Juliartawan,4,['SANG PUTU WIDIANA~Subsider Penjara (2 Bulan ...,1(satu) buah HP merk Realme warna hitam putih,Dakwaan\nKe Satu\n- Bahwa terdakwa SANG PUTU...,[SANG PUTU WIDIANA],[SANG PUTU WIDIANA~Subsider Penjara (2 Bulan )...
4,116/Pid.Sus/2024/PN Sgr,Narkotika,['WAYAN DARNA MASTONO'],"['Isnarti Jayaningsih, S.H.']",I Gusti Made Juliartawan,4,['WAYAN DARNA MASTONO~Subsider Penjara (2 Bula...,6 (enam) Paket plastik klip bening yang berisi...,dakwaan petugas Sat Narkoba Polres Buleleng m...,[WAYAN DARNA MASTONO],[WAYAN DARNA MASTONO~Subsider Penjara (2 Bulan...
...,...,...,...,...,...,...,...,...,...,...,...
1026,223/Pid.Sus/2017/PN Sgr,Perlindungan dan Pengelolaan Lingkungan Hidup,['Jumat Ariyanto'],"['I Nyoman Sulitra, SH.,MH.']",Ida Bagus Bama Dewa. P,3,['Jumat Ariyanto~Pidana Kurungan (1 Tahun )\nS...,1 (satu) 1 (satu) batang kayu balang-balang be...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...,[Jumat Ariyanto],[Jumat Ariyanto~Pidana Kurungan (1 Tahun )\nSu...
1027,224/Pid.B/2017/PN Sgr,Kejahatan Perjudian,['I KETUT ANA SAPUTRA Alias BENCUT'],"['MADE ASTINI, SH.']",Sudar,1,['I KETUT ANA SAPUTRA Alias BENCUT~Pidana Penj...,36 (tiga puluh enam) lembar kertas rekapan,Perbuatan terdakwa sebagaimana diatur dan dian...,[I KETUT ANA SAPUTRA Alias BENCUT],[I KETUT ANA SAPUTRA Alias BENCUT~Pidana Penja...
1028,220/Pid.B/2017/PN Sgr,Penganiayaan,['Sahabudin Alias Udin'],"['I KETUT KINDRA, SH']",Ni Luh Suantini,2,['Sahabudin Alias Udin~Pidana Penjara Waktu Te...,1. 1 (satu) buah tabung gas elpiji ukuran 3 (t...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...,[Sahabudin Alias Udin],[Sahabudin Alias Udin~Pidana Penjara Waktu Ter...
1029,219/Pid.B/2017/PN Sgr,Kejahatan yang Membahayakan Keamananan Umum Ba...,['Putu Kristian Damanta'],"['I GUSTI ARYA WIDNYANA, SH.']",A.A. Sagung Yuni Wulantrisna,4,['Putu Kristian Damanta~Pidana Penjara Waktu T...,-1 (satu)unit sepeda motor merk Yamaha Jupiter...,Pertama Pasal 187 ayat (1) KUHP\ndan\nKedua ...,[Putu Kristian Damanta],[Putu Kristian Damanta~Pidana Penjara Waktu Te...


In [366]:
print(len(df_clean))
print(df_clean['transformed_terdakwa'].apply(lambda x: isinstance(x, list)).sum())  # Should return number of rows with lists
print(df_clean['transformed_putusan_hukuman'].apply(lambda x: isinstance(x, list)).sum())  # Same for the other column


1031
1031
1031


In [367]:
# Check number of elements in each list
df_clean['terdakwa_len'] = df_clean['transformed_terdakwa'].apply(len)
df_clean['putusan_len'] = df_clean['transformed_putusan_hukuman'].apply(len)

# Display rows where the lengths are not equal
mismatched_rows = df_clean[df_clean['terdakwa_len'] != df_clean['putusan_len']]
print(mismatched_rows[['terdakwa_len', 'putusan_len']].head())


     terdakwa_len  putusan_len
18              2            3
382             2            1


In [368]:
print(mismatched_rows)

                 nomor_perkara          klasifikasi_perkara  \
18   93/Pid.Sus-LH/2024/PN Sgr  Konservasi Sumber Daya Alam   
382      147/Pid.B/2021/PN Sgr                    Pencurian   

                                              terdakwa  \
18   ['KETUT SUMANTRA alias LOTOT', 'MOCH HASAN BAS...   
382  ['IDA BAGUS ARI WIBAWA Als. GUS UNENG', 'KKADE...   

                    penuntut_umum                              hakim  \
18   ['Kadek Adi Pramarta, S.H.']                    I Made Bagiarta   
382   ['Ida Kade Widiatmika, SH']  Anak Agung Ngurah Budhi Dharmawan   

     jumlah_saksi                                    putusan_hukuman  \
18              4  ['MOCH HASAN BASRI alias BAS~Pidana Denda Rp.5...   
382             4  ['IDA BAGUS ARI WIBAWA Als. GUS UNENG~Pidana P...   

                                  cleaned_barang_bukti  \
18   1 (satu) pucuk senapan angin warna hitam rakit...   
382  1 (satu) buah laptop merk axioo warna hitam. 1...   

                          

In [369]:
df_clean = df_clean[df_clean['terdakwa_len'] == df_clean['putusan_len']]

In [390]:
def match_terdakwa_to_putusan(df):
    combined_terdakwa_putusan_list = []
    for index, row in df.iterrows():
        for terdakwa in row['transformed_terdakwa']:
            matched = False
            for putusan_hukuman in row['transformed_putusan_hukuman']:
                if terdakwa in putusan_hukuman:
                    temp_list = [row['nomor_perkara'], row['klasifikasi_perkara'], row['penuntut_umum'], row['hakim'], row['jumlah_saksi'], row['cleaned_barang_bukti'], row['cleaned_dakwaan'],terdakwa, putusan_hukuman] 
                    combined_terdakwa_putusan_list.append(temp_list)
                    matched = True
                    break
            if not matched:
                print(f'no match {terdakwa} at index {index}')
    df_combined = pd.DataFrame(combined_terdakwa_putusan_list, columns=['nomor_perkara', 'klasifikasi_perkara', 'penuntut_umum', 'hakim', 'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan', 'terdakwa', 'putusan_hukuman'])

    return df_combined

df_clean_exploded = match_terdakwa_to_putusan(df_clean)
print(df_clean_exploded)

                nomor_perkara  \
0       122/Pid.B/2024/PN Sgr   
1       122/Pid.B/2024/PN Sgr   
2     118/Pid.Sus/2024/PN Sgr   
3     114/Pid.Sus/2024/PN Sgr   
4     115/Pid.Sus/2024/PN Sgr   
...                       ...   
1195  223/Pid.Sus/2017/PN Sgr   
1196    224/Pid.B/2017/PN Sgr   
1197    220/Pid.B/2017/PN Sgr   
1198    219/Pid.B/2017/PN Sgr   
1199    216/Pid.B/2017/PN Sgr   

                                    klasifikasi_perkara  \
0                                   Kejahatan Perjudian   
1                                   Kejahatan Perjudian   
2                                             Narkotika   
3                                             Narkotika   
4                                             Narkotika   
...                                                 ...   
1195      Perlindungan dan Pengelolaan Lingkungan Hidup   
1196                                Kejahatan Perjudian   
1197                                       Penganiayaan   
1198  Kejahat

In [391]:
df_clean_exploded.iloc[1]['putusan_hukuman']

'KOMANG ADITYA~Pidana Penjara Waktu Tertentu (4 Bulan )'

In [392]:
type(df_clean_exploded.iloc[1]['putusan_hukuman'])

str

In [393]:
print(len(df_clean_exploded['terdakwa']))
print(len(df_clean_exploded['putusan_hukuman']))

1200
1200


In [394]:
for i in range(10):
    print(i, ".", df_clean_exploded.iloc[i]['putusan_hukuman'])

0 . KETUT SUBAGIA~Pidana Penjara Waktu Tertentu (4 Bulan )
1 . KOMANG ADITYA~Pidana Penjara Waktu Tertentu (4 Bulan )
2 . I NYOMAN SUARTA~Subsider Penjara (2 Bulan )
Pidana Denda Rp.1.000.000.000,00
Pidana Penjara Waktu Tertentu (4 Tahun 4 Bulan )
3 . RICO JAYADI~Subsider Penjara (3 Bulan )
Pidana Denda Rp.1.000.000.000,00
Pidana Penjara Waktu Tertentu (6 Tahun )
4 . SANG PUTU WIDIANA~Subsider Penjara (2 Bulan )
Pidana Denda Rp.800.000.000,00
Pidana Penjara Waktu Tertentu (4 Tahun )
5 . WAYAN DARNA MASTONO~Subsider Penjara (2 Bulan )
Pidana Penjara Waktu Tertentu (4 Tahun )
Pidana Denda Rp.800.000.000,00
6 . MADE SUDAMA ALIAS KARTOLO~Subsider Penjara (1 Tahun 2 Bulan )
7 . PUTU WISNU ALIAS WISNU~Subsider Penjara (10 Bulan )
8 . MUCH.ARIFIN~Pidana Penjara Waktu Tertentu (1 Tahun 4 Bulan )
9 . I KOMANG TEKEN Alias TEKEN~Pidana Penjara Waktu Tertentu (9 Bulan )


In [395]:
IXA = 90
print(df_clean_exploded.iloc[IXA]['putusan_hukuman'])
print(df_clean_exploded.iloc[IXA]['nomor_perkara'])

I MADE SENADIPA~Pidana Penjara Waktu Tertentu (2 Tahun 3 Bulan )
22/Pid.B/2024/PN Sgr


### Data transformation: Extract Multiple Penuntut Umum

In [445]:
# Get the indices of rows where any column contains NaN values
null_indices = df_clean_exploded[df_clean_exploded.isnull().any(axis=1)].index

# Display the indices of rows with NaN values
print(null_indices)

print(df_clean_exploded.iloc[890]['cleaned_barang_bukti'])
print(df_clean_exploded.iloc[890]['nomor_perkara'])


Index([890], dtype='int64')
nan
86/Pid.B/2019/PN Sgr


In [396]:
df_clean_exploded

,nomor_perkara,klasifikasi_perkara,penuntut_umum,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,terdakwa,putusan_hukuman
0,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN\n PERTAMA \n- Bahwa Terdakwa KETUT SU...,KETUT SUBAGIA,KETUT SUBAGIA~Pidana Penjara Waktu Tertentu (4...
1,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"['Kadek Adi Pramarta, S.H.']",I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN\n PERTAMA \n- Bahwa Terdakwa KETUT SU...,KOMANG ADITYA,KOMANG ADITYA~Pidana Penjara Waktu Tertentu (4...
2,118/Pid.Sus/2024/PN Sgr,Narkotika,"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN \nKESATU\n Bahwa ia Terdak...,I NYOMAN SUARTA,I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\nP...
3,114/Pid.Sus/2024/PN Sgr,Narkotika,"['I MADE SUTAPA,S.H.']",I Made Bagiarta,3,1 (satu) buah dompet warna hijau 20 (dua puluh...,DAKWAAN \nKESATU\n ...,RICO JAYADI,RICO JAYADI~Subsider Penjara (3 Bulan )\nPidan...
4,115/Pid.Sus/2024/PN Sgr,Narkotika,"['Isnarti Jayaningsih, S.H.']",I Gusti Made Juliartawan,4,1(satu) buah HP merk Realme warna hitam putih,Dakwaan\nKe Satu\n- Bahwa terdakwa SANG PUTU...,SANG PUTU WIDIANA,SANG PUTU WIDIANA~Subsider Penjara (2 Bulan )\...
...,...,...,...,...,...,...,...,...,...
1195,223/Pid.Sus/2017/PN Sgr,Perlindungan dan Pengelolaan Lingkungan Hidup,"['I Nyoman Sulitra, SH.,MH.']",Ida Bagus Bama Dewa. P,3,1 (satu) 1 (satu) batang kayu balang-balang be...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...,Jumat Ariyanto,Jumat Ariyanto~Pidana Kurungan (1 Tahun )\nSub...
1196,224/Pid.B/2017/PN Sgr,Kejahatan Perjudian,"['MADE ASTINI, SH.']",Sudar,1,36 (tiga puluh enam) lembar kertas rekapan,Perbuatan terdakwa sebagaimana diatur dan dian...,I KETUT ANA SAPUTRA Alias BENCUT,I KETUT ANA SAPUTRA Alias BENCUT~Pidana Penjar...
1197,220/Pid.B/2017/PN Sgr,Penganiayaan,"['I KETUT KINDRA, SH']",Ni Luh Suantini,2,1. 1 (satu) buah tabung gas elpiji ukuran 3 (t...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...,Sahabudin Alias Udin,Sahabudin Alias Udin~Pidana Penjara Waktu Tert...
1198,219/Pid.B/2017/PN Sgr,Kejahatan yang Membahayakan Keamananan Umum Ba...,"['I GUSTI ARYA WIDNYANA, SH.']",A.A. Sagung Yuni Wulantrisna,4,-1 (satu)unit sepeda motor merk Yamaha Jupiter...,Pertama Pasal 187 ayat (1) KUHP\ndan\nKedua ...,Putu Kristian Damanta,Putu Kristian Damanta~Pidana Penjara Waktu Ter...


In [397]:
df_clean_exploded['penuntut_umum']

0         ['Kadek Adi Pramarta, S.H.']
1         ['Kadek Adi Pramarta, S.H.']
2               ['I MADE SUTAPA,S.H.']
3               ['I MADE SUTAPA,S.H.']
4        ['Isnarti Jayaningsih, S.H.']
                     ...              
1195     ['I Nyoman Sulitra, SH.,MH.']
1196              ['MADE ASTINI, SH.']
1197            ['I KETUT KINDRA, SH']
1198    ['I GUSTI ARYA WIDNYANA, SH.']
1199              ['MADE ASTINI, SH.']
Name: penuntut_umum, Length: 1200, dtype: object

In [398]:
# Count unique data types in the column
unique_types = df_clean_exploded['penuntut_umum'].apply(type).unique()

# Display the unique types
print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'str'>]


In [399]:
df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [400]:
# Count unique data types in the column
unique_types = df_clean_exploded['penuntut_umum'].apply(type).unique()

# Display the unique types
print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'list'>]


In [401]:
# Count unique data types in the column
unique_types = df_clean_exploded['penuntut_umum'].apply(len).unique()

# Display the unique types
print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [1 2]


In [402]:
# Count the length of each list in the 'penuntut_umum' column
list_lengths = df_clean_exploded['penuntut_umum'].apply(len)

# Get unique list lengths
unique_lengths = list_lengths.unique()

# Print out the unique lengths and the corresponding indices
print(f"Unique lengths of data in the 'penuntut_umum' column: {unique_lengths}")
print("Indices for each unique length:")

# Display indices for each unique length
for length in unique_lengths:
    # Find indices where the list length matches the current length
    indices = list_lengths[list_lengths == length].index
    print(f"Length {length}: Indices - {indices.tolist()}")


Unique lengths of data in the 'penuntut_umum' column: [1 2]
Indices for each unique length:
Length 1: Indices - [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 163, 164, 165, 166, 167, 169, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 20

In [403]:
for index, row in df_clean_exploded.iterrows():
    data = row['penuntut_umum']
    if len(data) > 1:
        print(index, row['penuntut_umum'])


156 ['Komang Tirtawati, S.H.', 'KOMANG TIRTA WATI, S.H.']
162 ['IDA KADE WIDIATMIKA, SH', 'IDA KADE WIDIATMIKA, SH']
168 ['IDA KADE WIDIATMIKA, SH', 'IDA KADE WIDIATMIKA, SH']
170 ['IDA KADE WIDIATMIKA, SH', 'IDA KADE WIDIATMIKA, SH']
232 ['ISNARTI JAYANINGSIH, SH.', 'I MADE HERI PERMANA PUTRA,SH.']
995 ['PUTU ANDY SUTADHARMA, SH.', 'Gusti Putu Karmawan, S.H.']


In [404]:
# df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: list(set([name.lower() for name in x])))
df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: list(set(x)))
df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: sorted(x))

In [405]:
for index, row in df_clean_exploded.iterrows():
    data = row['penuntut_umum']
    if len(data) > 1:
        print(index, row['penuntut_umum'])

156 ['KOMANG TIRTA WATI, S.H.', 'Komang Tirtawati, S.H.']
232 ['I MADE HERI PERMANA PUTRA,SH.', 'ISNARTI JAYANINGSIH, SH.']
995 ['Gusti Putu Karmawan, S.H.', 'PUTU ANDY SUTADHARMA, SH.']


In [406]:
# Explode the 'penuntut_umum' column so that each value in the list gets its own row
df_clean_penuntut_exploded = df_clean_exploded.explode('penuntut_umum', ignore_index=True)


In [407]:
df_clean_penuntut_exploded

,nomor_perkara,klasifikasi_perkara,penuntut_umum,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,terdakwa,putusan_hukuman
0,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"Kadek Adi Pramarta, S.H.",I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN\n PERTAMA \n- Bahwa Terdakwa KETUT SU...,KETUT SUBAGIA,KETUT SUBAGIA~Pidana Penjara Waktu Tertentu (4...
1,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"Kadek Adi Pramarta, S.H.",I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN\n PERTAMA \n- Bahwa Terdakwa KETUT SU...,KOMANG ADITYA,KOMANG ADITYA~Pidana Penjara Waktu Tertentu (4...
2,118/Pid.Sus/2024/PN Sgr,Narkotika,"I MADE SUTAPA,S.H.",I Made Bagiarta,3,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN \nKESATU\n Bahwa ia Terdak...,I NYOMAN SUARTA,I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\nP...
3,114/Pid.Sus/2024/PN Sgr,Narkotika,"I MADE SUTAPA,S.H.",I Made Bagiarta,3,1 (satu) buah dompet warna hijau 20 (dua puluh...,DAKWAAN \nKESATU\n ...,RICO JAYADI,RICO JAYADI~Subsider Penjara (3 Bulan )\nPidan...
4,115/Pid.Sus/2024/PN Sgr,Narkotika,"Isnarti Jayaningsih, S.H.",I Gusti Made Juliartawan,4,1(satu) buah HP merk Realme warna hitam putih,Dakwaan\nKe Satu\n- Bahwa terdakwa SANG PUTU...,SANG PUTU WIDIANA,SANG PUTU WIDIANA~Subsider Penjara (2 Bulan )\...
...,...,...,...,...,...,...,...,...,...
1198,223/Pid.Sus/2017/PN Sgr,Perlindungan dan Pengelolaan Lingkungan Hidup,"I Nyoman Sulitra, SH.,MH.",Ida Bagus Bama Dewa. P,3,1 (satu) 1 (satu) batang kayu balang-balang be...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...,Jumat Ariyanto,Jumat Ariyanto~Pidana Kurungan (1 Tahun )\nSub...
1199,224/Pid.B/2017/PN Sgr,Kejahatan Perjudian,"MADE ASTINI, SH.",Sudar,1,36 (tiga puluh enam) lembar kertas rekapan,Perbuatan terdakwa sebagaimana diatur dan dian...,I KETUT ANA SAPUTRA Alias BENCUT,I KETUT ANA SAPUTRA Alias BENCUT~Pidana Penjar...
1200,220/Pid.B/2017/PN Sgr,Penganiayaan,"I KETUT KINDRA, SH",Ni Luh Suantini,2,1. 1 (satu) buah tabung gas elpiji ukuran 3 (t...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...,Sahabudin Alias Udin,Sahabudin Alias Udin~Pidana Penjara Waktu Tert...
1201,219/Pid.B/2017/PN Sgr,Kejahatan yang Membahayakan Keamananan Umum Ba...,"I GUSTI ARYA WIDNYANA, SH.",A.A. Sagung Yuni Wulantrisna,4,-1 (satu)unit sepeda motor merk Yamaha Jupiter...,Pertama Pasal 187 ayat (1) KUHP\ndan\nKedua ...,Putu Kristian Damanta,Putu Kristian Damanta~Pidana Penjara Waktu Ter...


### Data Transformation: Sentences to Month

In [408]:
pattern = r"Pidana Penjara Waktu Tertentu\s*\(\s*(\d+)\s*(Tahun)?\s*(\d*)\s*(Bulan)?\s*\)"
pattern_2 = r"Subsider Penjara\s*\(\s*(\d+)\s*(Tahun)?\s*(\d*)\s*(Bulan)?\s*\)"
pattern_3 = r"Subsider Kurungan\s*\(\s*(\d+)\s*(Tahun)?\s*(\d*)\s*(Bulan)?\s*\)"

for index, row in df_clean_penuntut_exploded.iterrows():
    # Use re.search to find the match
    text = row["putusan_hukuman"]
    match = re.search(pattern, text, re.IGNORECASE)
    match_2 = re.search(pattern_2, text, re.IGNORECASE)
    match_3 = re.search(pattern_3, text, re.IGNORECASE)

    # Extract and print the result
    if match:
        extracted_duration = match.group(0)
        print(index,"-", extracted_duration)

    elif match_2:
        extracted_duration = match_2.group(0)
        print(index,"-", extracted_duration)

    elif match_3:
        extracted_duration = match_3.group(0)
        print(index,"-", extracted_duration)
        
    else:
        print(f"{index} - No match found.")

0 - Pidana Penjara Waktu Tertentu (4 Bulan )
1 - Pidana Penjara Waktu Tertentu (4 Bulan )
2 - Pidana Penjara Waktu Tertentu (4 Tahun 4 Bulan )
3 - Pidana Penjara Waktu Tertentu (6 Tahun )
4 - Pidana Penjara Waktu Tertentu (4 Tahun )
5 - Pidana Penjara Waktu Tertentu (4 Tahun )
6 - Subsider Penjara (1 Tahun 2 Bulan )
7 - Subsider Penjara (10 Bulan )
8 - Pidana Penjara Waktu Tertentu (1 Tahun 4 Bulan )
9 - Pidana Penjara Waktu Tertentu (9 Bulan )
10 - Pidana Penjara Waktu Tertentu (9 Bulan )
11 - Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )
12 - Pidana Penjara Waktu Tertentu (6 Bulan )
13 - Pidana Penjara Waktu Tertentu (1 Tahun )
14 - Pidana Penjara Waktu Tertentu (9 Bulan )
15 - Pidana Penjara Waktu Tertentu (9 Bulan )
16 - Pidana Penjara Waktu Tertentu (9 Bulan )
17 - Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )
18 - Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )
19 - Pidana Penjara Waktu Tertentu (1 Tahun )
20 - Pidana Penjara Waktu Tertentu (9 Bulan )
21 - Pidana Penjara W

In [431]:
pattern = r"Pidana Penjara Waktu Tertentu\s*\(\s*(\d+)\s*(Tahun)?\s*(\d*)\s*(Bulan)?\s*\)"
pattern_2 = r"Subsider Penjara\s*\(\s*(\d+)\s*(Tahun)?\s*(\d*)\s*(Bulan)?\s*\)"
pattern_3 = r"Subsider Kurungan\s*\(\s*(\d+)\s*(Tahun)?\s*(\d*)\s*(Bulan)?\s*\)"
pattern_4 = r"Pidana Kurungan\s*\(\s*(\d+)\s*(Tahun)?\s*(\d*)\s*(Bulan)?\s*\)"

# pattern_5 = r"Pidana Penjara Waktu Tertentu\s*\(\s*(\d+)\s*(Bulan)?\s*(\d*)\s*(Hari)?\s*\)"
# pattern_6 = r"Subsider Penjara\s*\(\s*(\d+)\s*(Bulan)?\s*(\d*)\s*(Hari)?\s*\)"
# pattern_7 = r"Subsider Kurungan\s*\(\s*(\d+)\s*(Bulan)?\s*(\d*)\s*(Hari)?\s*\)"
# pattern_8 = r"Pidana Kurungan\s*\(\s*(\d+)\s*(Bulan)?\s*(\d*)\s*(Hari)?\s*\)"

pattern_5 = r"~Pidana Penjara Waktu Tertentu\s*\(\s*(\d+)\s*(\d*)\s*(Tahun)?\s*(Bulan)?\s*(\d*)\s*(Hari)?\s*\)"
pattern_6 = r"~Subsider Penjara\s*\(\s*(\d+)\s*(\d*)\s*(Tahun)?\s*(Bulan)?\s*(\d*)\s*(Hari)?\s*\)"
pattern_7 = r"~Subsider Kurungan\s*\(\s*(\d+)\s*(\d*)\s*(Tahun)?\s*(Bulan)?\s*(\d*)\s*(Hari)?\s*\)"
pattern_8 = r"~Pidana Kurungan\s*\(\s*(\d+)\s*(\d*)\s*(Tahun)?\s*(Bulan)?\s*(\d*)\s*(Hari)?\s*\)"

extracted_data = []

for index, row in df_clean_penuntut_exploded.iterrows():
    text = row["putusan_hukuman"]
    match = re.search(pattern, text, re.IGNORECASE)
    match_2 = re.search(pattern_2, text, re.IGNORECASE)
    match_3 = re.search(pattern_3, text, re.IGNORECASE)
    match_4 = re.search(pattern_4, text, re.IGNORECASE)

    match_5 = re.search(pattern_5, text, re.IGNORECASE)
    match_6 = re.search(pattern_6, text, re.IGNORECASE)
    match_7 = re.search(pattern_7, text, re.IGNORECASE)
    match_8 = re.search(pattern_8, text, re.IGNORECASE)

    if match:
        extracted_duration = match.group(0)
    elif match_2:
        extracted_duration = match_2.group(0)
    elif match_3:
        extracted_duration = match_3.group(0)
    elif match_4:
        extracted_duration = match_4.group(0)
    elif match_5:
        extracted_duration = match_5.group(0)
    elif match_6:
        extracted_duration = match_6.group(0)
    elif match_7:
        extracted_duration = match_7.group(0)
    elif match_8:
        extracted_duration = match_8.group(0)
    else:
        extracted_duration = None
    
    print(index, extracted_duration)
    df_clean_penuntut_exploded.loc[index,'extracted_duration'] = extracted_duration



0 Pidana Penjara Waktu Tertentu (4 Bulan )
1 Pidana Penjara Waktu Tertentu (4 Bulan )
2 Pidana Penjara Waktu Tertentu (4 Tahun 4 Bulan )
3 Pidana Penjara Waktu Tertentu (6 Tahun )
4 Pidana Penjara Waktu Tertentu (4 Tahun )
5 Pidana Penjara Waktu Tertentu (4 Tahun )
6 Subsider Penjara (1 Tahun 2 Bulan )
7 Subsider Penjara (10 Bulan )
8 Pidana Penjara Waktu Tertentu (1 Tahun 4 Bulan )
9 Pidana Penjara Waktu Tertentu (9 Bulan )
10 Pidana Penjara Waktu Tertentu (9 Bulan )
11 Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )
12 Pidana Penjara Waktu Tertentu (6 Bulan )
13 Pidana Penjara Waktu Tertentu (1 Tahun )
14 Pidana Penjara Waktu Tertentu (9 Bulan )
15 Pidana Penjara Waktu Tertentu (9 Bulan )
16 Pidana Penjara Waktu Tertentu (9 Bulan )
17 Pidana Penjara Waktu Tertentu (1 Tahun 6 Bulan )
18 Pidana Penjara Waktu Tertentu (1 Tahun 2 Bulan )
19 Pidana Penjara Waktu Tertentu (1 Tahun )
20 Pidana Penjara Waktu Tertentu (9 Bulan )
21 Pidana Penjara Waktu Tertentu (1 Tahun 3 Bulan )
22 Pidana P

In [432]:
df_clean_penuntut_exploded.loc[1199, 'putusan_hukuman']

'I KETUT ANA SAPUTRA Alias BENCUT~Pidana Penjara Waktu Tertentu (4 Bulan 25 Hari)'

Not Catching Pidana Penjara Waktu Tertentu dengan Tahun Bulan dan Hari

In [434]:
# Filter rows where 'extracted_duration' is null
null_rows = df_clean_penuntut_exploded[df_clean_penuntut_exploded["extracted_duration"].isnull()]

# Display rows with null values in 'extracted_duration'
print(null_rows['putusan_hukuman'])

null_rows

611                   NYOMAN SUKRIATMAJA~Pidana Bersyarat
612                          Wayan Lanus~Pidana Bersyarat
1072    Ketut Bagus Jolinda Atmaja Alias Jolinda~Pidan...
1082    Nyoman Sudiarta alias Comek~Pidana Penjara Wak...
1115    Gusti Ngurah Darma Putra Alias Cuplis~Pidana B...
1116    Gusti Putu Abdiyasa Alias Gusti Abdi~Pidana Be...
Name: putusan_hukuman, dtype: object


,nomor_perkara,klasifikasi_perkara,penuntut_umum,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,terdakwa,putusan_hukuman,extracted_duration
611,172/Pid.B/2020/PN Sgr,Penganiayaan,"MADE JUNI ARTINI, S.H.",I Gede Karang Anggayasa,4,- 1 (satu) buah kayu (potongan alu) - 1 ...,Bahwa ia terdakwa NYOMAN SUKRIATMAJA pada hari...,NYOMAN SUKRIATMAJA,NYOMAN SUKRIATMAJA~Pidana Bersyarat,None
612,170/Pid.B/2020/PN Sgr,Penganiayaan,"I MADE HERI PERMANA PUTRA,SH.",I Gede Karang Anggayasa,4,satu buah pedang beserta sarungnya dirampas un...,Bahwa ia Terdakwa Wayan Lanus pada hari Kamis ...,Wayan Lanus,Wayan Lanus~Pidana Bersyarat,None
1072,139/Pid.B/2018/PN Sgr,Penggelapan,"MADE JUNI ARTINI, S.H.",Sudar,2,----------------------------------------------...,- Bahwa terdakwa KETUT BAGUS JOLINDA ATMAJA pa...,Ketut Bagus Jolinda Atmaja Alias Jolinda,Ketut Bagus Jolinda Atmaja Alias Jolinda~Pidan...,None
1082,123/Pid.B/2018/PN Sgr,Kejahatan Perjudian,"I KETUT KINDRA, SH",Sudar,3,1 (satu) bendel Kupon putih berisi tulisan ang...,"dakwaan, terdakwa NYOMAN SUDIARTA Als COMEK y...",Nyoman Sudiarta alias Comek,Nyoman Sudiarta alias Comek~Pidana Penjara Wak...,None
1115,82/Pid.B/2018/PN Sgr,Kejahatan Terhadap Ketertiban Umum,"I GEDE PUTU ASTAWA, SH.",Sudar,3,1(satu) buah baju kaos warna hitam milik korba...,"dakwaan tersebut diatas, yaitu berawal kesalah...",Gusti Ngurah Darma Putra Alias Cuplis,Gusti Ngurah Darma Putra Alias Cuplis~Pidana B...,None
1116,82/Pid.B/2018/PN Sgr,Kejahatan Terhadap Ketertiban Umum,"I GEDE PUTU ASTAWA, SH.",Sudar,3,1(satu) buah baju kaos warna hitam milik korba...,"dakwaan tersebut diatas, yaitu berawal kesalah...",Gusti Putu Abdiyasa Alias Gusti Abdi,Gusti Putu Abdiyasa Alias Gusti Abdi~Pidana Be...,None


In [436]:
#TEMPORARY
df_clean_penuntut_exploded = df_clean_penuntut_exploded.dropna(subset=['extracted_duration'])

In [438]:
df_extracted = df_clean_penuntut_exploded.copy()

pattern_0 = r"\(?\s*(\d+)?\s*Tahun?\s*(\d+)?\s*Bulan?\s*(\d+)\s*Hari?\s*\)?"
pattern = r"\(?\s*(\d+)?\s*Tahun?\s*(\d+)?\s*Bulan?\s*\)?"
pattern_tahun = r"\s*(\d+)\s*Tahun"
pattern_bulan = r"\s*(\d+)\s*Bulan"

# Initialize new columns in the DataFrame for years and months
df_extracted['Tahun'] = None
df_extracted['Bulan'] = None
df_extracted['Hari'] = None

# Iterate over each row and extract years and months
for index, row in df_extracted.iterrows():
    text = row["putusan_hukuman"]
    match = re.search(pattern, text, re.IGNORECASE)
    match_2 = re.search(pattern_tahun, text, re.IGNORECASE)
    match_3 = re.search(pattern_bulan, text, re.IGNORECASE)
    match_0 = re.search(pattern_0, text, re.IGNORECASE)
    
    if match:
        tahun = int(match.group(1)) if match.group(1) else 0  # Extract "Tahun" as an integer
        bulan = int(match.group(2)) if match.group(2) else 0  # Extract "Bulan" as an integer, default to 0 if empty

    elif match_0:
        tahun = int(match.group(1))  # Extract "Tahun" as an integer
        bulan = int(match.group(2)) if match.group(2) else 0  # Extract "Bulan" as an integer, default to 0 if empty
        hari = int(match.group(3)) if match.group(3) else 0

    elif match_2:
        tahun = int(match_2.group(1))
        bulan = 0
        hari = 0

    elif match_3:
        tahun = 0
        bulan = int(match_3.group(1))
        hari = 0

    else:
        tahun = None
        bulan = None
        hari = None

    df_extracted.loc[index, 'Tahun'] = tahun
    df_extracted.loc[index, 'Bulan'] = bulan
    df_extracted.loc[index, 'Hari'] = hari
# Display the updated DataFrame
print(df_extracted)

                nomor_perkara  \
0       122/Pid.B/2024/PN Sgr   
1       122/Pid.B/2024/PN Sgr   
2     118/Pid.Sus/2024/PN Sgr   
3     114/Pid.Sus/2024/PN Sgr   
4     115/Pid.Sus/2024/PN Sgr   
...                       ...   
1198  223/Pid.Sus/2017/PN Sgr   
1199    224/Pid.B/2017/PN Sgr   
1200    220/Pid.B/2017/PN Sgr   
1201    219/Pid.B/2017/PN Sgr   
1202    216/Pid.B/2017/PN Sgr   

                                    klasifikasi_perkara  \
0                                   Kejahatan Perjudian   
1                                   Kejahatan Perjudian   
2                                             Narkotika   
3                                             Narkotika   
4                                             Narkotika   
...                                                 ...   
1198      Perlindungan dan Pengelolaan Lingkungan Hidup   
1199                                Kejahatan Perjudian   
1200                                       Penganiayaan   
1201  Kejahat

In [439]:
# Check if any column contains NaN values
df_extracted.isnull().any()

nomor_perkara           False
klasifikasi_perkara     False
penuntut_umum           False
hakim                   False
jumlah_saksi            False
cleaned_barang_bukti     True
cleaned_dakwaan         False
terdakwa                False
putusan_hukuman         False
extracted_duration      False
Tahun                   False
Bulan                   False
Hari                    False
dtype: bool

In [480]:
# Check for NaN values in 'extracted_duration' and drop the rows
df_extracted = df_extracted.dropna(subset=['extracted_duration'])
df_extracted = df_extracted.dropna(subset=['cleaned_barang_bukti']) # TEMPORARY


In [481]:
# Check if any column contains NaN values
df_extracted.isnull().any()

nomor_perkara           False
klasifikasi_perkara     False
penuntut_umum           False
hakim                   False
jumlah_saksi            False
cleaned_barang_bukti    False
cleaned_dakwaan         False
terdakwa                False
putusan_hukuman         False
extracted_duration      False
Tahun                   False
Bulan                   False
Hari                    False
dtype: bool

In [483]:
df_extracted['total_pidana_penjara_bulan'] = (df_extracted['Tahun'] * 12 + df_extracted['Bulan'].fillna(0).astype(int)).astype(int)

C:\Users\Kalea\AppData\Local\Temp\ipykernel_25740\503598288.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_extracted['total_pidana_penjara_bulan'] = (df_extracted['Tahun'] * 12 + df_extracted['Bulan'].fillna(0).astype(int)).astype(int)


In [484]:
df_extracted

,nomor_perkara,klasifikasi_perkara,penuntut_umum,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,terdakwa,putusan_hukuman,extracted_duration,Tahun,Bulan,Hari,total_pidana_penjara_bulan
0,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"Kadek Adi Pramarta, S.H.",I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN\n PERTAMA \n- Bahwa Terdakwa KETUT SU...,KETUT SUBAGIA,KETUT SUBAGIA~Pidana Penjara Waktu Tertentu (4...,Pidana Penjara Waktu Tertentu (4 Bulan ),0,4,0,4
1,122/Pid.B/2024/PN Sgr,Kejahatan Perjudian,"Kadek Adi Pramarta, S.H.",I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN\n PERTAMA \n- Bahwa Terdakwa KETUT SU...,KOMANG ADITYA,KOMANG ADITYA~Pidana Penjara Waktu Tertentu (4...,Pidana Penjara Waktu Tertentu (4 Bulan ),0,4,0,4
2,118/Pid.Sus/2024/PN Sgr,Narkotika,"I MADE SUTAPA,S.H.",I Made Bagiarta,3,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN \nKESATU\n Bahwa ia Terdak...,I NYOMAN SUARTA,I NYOMAN SUARTA~Subsider Penjara (2 Bulan )\nP...,Pidana Penjara Waktu Tertentu (4 Tahun 4 Bulan ),4,4,0,52
3,114/Pid.Sus/2024/PN Sgr,Narkotika,"I MADE SUTAPA,S.H.",I Made Bagiarta,3,1 (satu) buah dompet warna hijau 20 (dua puluh...,DAKWAAN \nKESATU\n ...,RICO JAYADI,RICO JAYADI~Subsider Penjara (3 Bulan )\nPidan...,Pidana Penjara Waktu Tertentu (6 Tahun ),6,0,0,72
4,115/Pid.Sus/2024/PN Sgr,Narkotika,"Isnarti Jayaningsih, S.H.",I Gusti Made Juliartawan,4,1(satu) buah HP merk Realme warna hitam putih,Dakwaan\nKe Satu\n- Bahwa terdakwa SANG PUTU...,SANG PUTU WIDIANA,SANG PUTU WIDIANA~Subsider Penjara (2 Bulan )\...,Pidana Penjara Waktu Tertentu (4 Tahun ),4,0,0,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198,223/Pid.Sus/2017/PN Sgr,Perlindungan dan Pengelolaan Lingkungan Hidup,"I Nyoman Sulitra, SH.,MH.",Ida Bagus Bama Dewa. P,3,1 (satu) 1 (satu) batang kayu balang-balang be...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...,Jumat Ariyanto,Jumat Ariyanto~Pidana Kurungan (1 Tahun )\nSub...,Subsider Kurungan (3 Bulan ),1,0,0,12
1199,224/Pid.B/2017/PN Sgr,Kejahatan Perjudian,"MADE ASTINI, SH.",Sudar,1,36 (tiga puluh enam) lembar kertas rekapan,Perbuatan terdakwa sebagaimana diatur dan dian...,I KETUT ANA SAPUTRA Alias BENCUT,I KETUT ANA SAPUTRA Alias BENCUT~Pidana Penjar...,~Pidana Penjara Waktu Tertentu (4 Bulan 25 Hari),0,4,0,4
1200,220/Pid.B/2017/PN Sgr,Penganiayaan,"I KETUT KINDRA, SH",Ni Luh Suantini,2,1. 1 (satu) buah tabung gas elpiji ukuran 3 (t...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...,Sahabudin Alias Udin,Sahabudin Alias Udin~Pidana Penjara Waktu Tert...,Pidana Penjara Waktu Tertentu (9 Bulan ),0,9,0,9
1201,219/Pid.B/2017/PN Sgr,Kejahatan yang Membahayakan Keamananan Umum Ba...,"I GUSTI ARYA WIDNYANA, SH.",A.A. Sagung Yuni Wulantrisna,4,-1 (satu)unit sepeda motor merk Yamaha Jupiter...,Pertama Pasal 187 ayat (1) KUHP\ndan\nKedua ...,Putu Kristian Damanta,Putu Kristian Damanta~Pidana Penjara Waktu Ter...,Pidana Penjara Waktu Tertentu (2 Tahun ),2,0,0,24


In [487]:
df_extracted.to_csv('../Data/STAGE 2 PREPROCESSING/STAGE_2_PREPROCESSING_{database_name}.csv', index=False)

## Data Validation

In [489]:
df_extracted = pd.read_csv('../Data/STAGE 2 PREPROCESSING/STAGE_2_PREPROCESSING_{database_name}.csv')

selected_data = ['klasifikasi_perkara', 'terdakwa', 'hakim', 'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan', 'total_pidana_penjara_bulan']
df_extracted = df_extracted[selected_data].copy()

df_extracted.head(10)

,klasifikasi_perkara,terdakwa,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,total_pidana_penjara_bulan
0,Kejahatan Perjudian,KETUT SUBAGIA,I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN\n PERTAMA \n- Bahwa Terdakwa KETUT SU...,4
1,Kejahatan Perjudian,KOMANG ADITYA,I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN\n PERTAMA \n- Bahwa Terdakwa KETUT SU...,4
2,Narkotika,I NYOMAN SUARTA,I Made Bagiarta,3,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN \nKESATU\n Bahwa ia Terdak...,52
3,Narkotika,RICO JAYADI,I Made Bagiarta,3,1 (satu) buah dompet warna hijau 20 (dua puluh...,DAKWAAN \nKESATU\n ...,72
4,Narkotika,SANG PUTU WIDIANA,I Gusti Made Juliartawan,4,1(satu) buah HP merk Realme warna hitam putih,Dakwaan\nKe Satu\n- Bahwa terdakwa SANG PUTU...,48
5,Narkotika,WAYAN DARNA MASTONO,I Gusti Made Juliartawan,4,6 (enam) Paket plastik klip bening yang berisi...,dakwaan petugas Sat Narkoba Polres Buleleng m...,48
6,Narkotika,MADE SUDAMA ALIAS KARTOLO,Made Hermayanti Muliartha,3,1 (satu) buah plastik klip bening berisi butir...,DAKWAAN \nKESATU\n Bahwa ia Terdakw...,14
7,Narkotika,PUTU WISNU ALIAS WISNU,Made Hermayanti Muliartha,3,1 (satu) buah plastik klip bening berisi butir...,DAKWAAN \nKESATU\n Bahwa ia Terdakw...,10
8,Narkotika,MUCH.ARIFIN,I Gusti Made Juliartawan,3,1 (satu) pipet plastik didalamnya terdapat 2 (...,DAKWAAN\nKesatu \n-Bahwa Terdakwa MUCH ARIFIN...,16
9,Narkotika,I KOMANG TEKEN Alias TEKEN,I Made Bagiarta,4,1. 1 (satu) buah buah pipet kaca berisi residu...,KESATU\nBahwa Terdakwa 1. I Komang Teken Alias...,9


In [516]:
def validate_schema(df):
    print("**Schema Validation**")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Data types:\n{df.dtypes}\n")

def check_nulls(df):
    print("**Null Value Analysis**")
    null_counts = df.isnull().sum()
    print(f"Null counts per column:\n{null_counts}\n")

def check_duplicates(df):
    print("**Duplicate Rows**")
    duplicates = df[df.duplicated()]
    print(f"Number of duplicate rows: {len(duplicates)}\n")
    if not duplicates.empty:
        print(f"Duplicate rows:\n{duplicates}\n")

def summarize_statistics(df):
    print("**Statistics Summary**")
    stats = df.describe(include="all").transpose()
    print(tabulate(stats, headers='keys', tablefmt='pretty'))
    


In [ ]:
validate_schema(df_extracted)

**Schema Validation**
Columns: ['klasifikasi_perkara', 'terdakwa', 'hakim', 'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan', 'total_pidana_penjara_bulan']
Data types:
klasifikasi_perkara           object
terdakwa                      object
hakim                         object
jumlah_saksi                   int64
cleaned_barang_bukti          object
cleaned_dakwaan               object
total_pidana_penjara_bulan     int64
dtype: object



In [508]:
# Iterate through each column in the DataFrame
for column in df_extracted.columns:
    # Get unique data types in the column
    types_in_column = df_extracted[column].apply(type).unique()
    # Print the column name and its unique data types
    print(f"Data types in column '{column}' -> {types_in_column}")


Data types in column 'klasifikasi_perkara' -> [<class 'str'>]
Data types in column 'terdakwa' -> [<class 'str'>]
Data types in column 'hakim' -> [<class 'str'>]
Data types in column 'jumlah_saksi' -> [<class 'int'>]
Data types in column 'cleaned_barang_bukti' -> [<class 'str'>]
Data types in column 'cleaned_dakwaan' -> [<class 'str'>]
Data types in column 'total_pidana_penjara_bulan' -> [<class 'int'>]


In [509]:
check_nulls(df_extracted)

**Null Value Analysis**
Null counts per column:
klasifikasi_perkara           0
terdakwa                      0
hakim                         0
jumlah_saksi                  0
cleaned_barang_bukti          0
cleaned_dakwaan               0
total_pidana_penjara_bulan    0
dtype: int64



In [517]:
check_duplicates(df_extracted)

**Duplicate Rows**
Number of duplicate rows: 3

Duplicate rows:
    klasifikasi_perkara                             terdakwa  \
157         Penggelapan              KOMANG AGUS SULENDRAWAN   
234           Narkotika  I GEDE EKA ANGGA WIJANA alias ANGGA   
995          Pembunuhan                  I Ketut Budi Astawa   

                        hakim  jumlah_saksi  \
157  I Gusti Made Juliartawan             3   
234           I Made Bagiarta             3   
995          Mayasari Oktavia             6   

                                  cleaned_barang_bukti  \
157  1 (satu) unit Sepeda motor Jenis Honda BEAT, T...   
234  2 (dua) paket gulungan aluminium foil warna si...   
995  1 (satu) potong baju kaos warna hitam beri nod...   

                                       cleaned_dakwaan  \
157  - Bahwa ia Terdakwa KOMANG AGUS SULENDRAWAN Al...   
234  Bahwa  terdakwa  I GEDE EKA ANGGA WIJANA alias...   
995  Primair  \n \n-Bahwa ia terdakwa I KETUT BUDI ...   

     total_pidana_penjar

In [512]:
row1 = df.loc[157]
row2 = df.loc[234]
print(row1.equals(row2))


False


In [513]:
differences = row1 != row2
print(differences[differences]) 


nomor_perkara          True
klasifikasi_perkara    True
tanggal_pendaftaran    True
lama_proses            True
terdakwa               True
penuntut_umum          True
hakim                  True
jumlah_saksi           True
putusan_hukuman        True
barang_bukti           True
dakwaan                True
dtype: bool


In [518]:
summarize_statistics(df_extracted)

**Statistics Summary**
+----------------------------+--------+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+--------------------+--------------------+-----+-----+------+------+-------+
|                            | count  | unique |                                                                                   top                                                                                    | freq |        mean        |        std         | min | 25% | 50%  | 75%  |  max  |
+----------------------------+--------+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+--------------------+--------------------+-----+-----+------+------+-------+
|    klasifikasi_perkara     |  1196  |   33   |                    

In [ ]:
validation_results = pd.DataFrame()

validation_results = df_extracted.select_dtypes(include="number").apply(
    lambda x: x[(x < x.quantile(0.01)) | (x > x.quantile(0.99))]
)

print(tabulate(validation_results, headers='keys', tablefmt='pretty'))

+------+--------------+----------------------------+
|      | jumlah_saksi | total_pidana_penjara_bulan |
+------+--------------+----------------------------+
|  93  |     nan      |           144.0            |
|  96  |     15.0     |            nan             |
| 113  |     nan      |           144.0            |
| 144  |     nan      |           168.0            |
| 146  |     13.0     |            nan             |
| 220  |     nan      |           156.0            |
| 263  |     12.0     |            nan             |
| 279  |     nan      |           144.0            |
| 280  |     nan      |           144.0            |
| 448  |     13.0     |            nan             |
| 449  |     13.0     |            nan             |
| 450  |     13.0     |            nan             |
| 480  |     13.0     |            nan             |
| 563  |     12.0     |           156.0            |
| 743  |     nan      |           720.0            |
| 859  |     nan      |           168.0       

# Exploratory Data Analysis

# Text Summarization

In [110]:
df_model.loc[0,'cleaned_dakwaan']

'DAKWAAN\n  PERTAMA \n- Bahwa Terdakwa KETUT SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada hari Minggu tanggal 02 Juni 2024 sekitar jam 00.30 Wita atau pada suatu waktu dalam bulan Juni 2024 atau setidak-tidaknya dalam tahun 2024 bertempat di pesisir pantai Gondol, Desa Penyabangan, Kec. Gerokgak, Kab. Buleleng, Prov. Bali, atau setidak-tidaknya di suatu tempat yang masih termasuk dalam daerah hukum Pengadilan Negeri Singaraja, dengan sengaja menawarkan atau memberi kesempatan untuk permainan judi yaitu judi bola adil dan menjadikan sebagai pencaharian, atau dengan sengaja turut serta dalam suatu perusahaan untuk itu, yang dilakukan antara lain dengan cara sebagai berikut   -\nBahwa pada waktu dan tempat seperti tersebut diatas, pada saat saksi Polisi Dandik Danny Saputra dan saksi I Made Yogi Mahendra melakukan penyelidikan Pesisir Pantai Gerokgak rawan terjadi adanya tindak pidana, selanjutnya melakukan penangkapan terhadap Terdakwa KETUT SUBAGIA bersama dengan Terdakwa KOMANG A

In [111]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("CUDA version:", torch.version.cuda)


CUDA available: True
Device count: 1
CUDA version: 12.1


In [112]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.3092, 0.4637, 0.5389],
        [0.6815, 0.9689, 0.0069],
        [0.7541, 0.0938, 0.3127],
        [0.8638, 0.1901, 0.0217],
        [0.0092, 0.0146, 0.3554]])


In [113]:
# !nvcc --version

In [114]:
# !nvidia-smi

In [171]:
from transformers import pipeline

# pipe = pipeline("text2text-generation", model="facebook/mbart-large-50", use_fast=False, device="cuda", batch_size=2)
# pipe = pipeline("text2text-generation", model="indobenchmark/indobart-large")
# pipe = pipeline("summarization", model="mrm8488/t5-base-finetuned-indonesian-summarization")
# pipe = pipeline("text2text-generation", model="google/mt5-large", use_fast=False)

# tensorflow_summarization_models = [
#     "thonyyy/pegasus_indonesian_base-finetune",
#     "thonyyy/pegasus_indonesian_base-pretrain"
# ]

pytorch_summarization_models = [
    # "cahya/bert2bert-indonesian-summarization",
    "cahya/t5-base-indonesian-summarization-cased",
    "panggi/t5-small-indonesian-summarization-cased",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3",
    # "cahya/bert2gpt-indonesian-summarization",
    "panggi/t5-base-indonesian-summarization-cased",
    "rayendito/mt5-small-finetuned-xl-sum-indonesia",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs",
    # "rowjak/bert-indonesian-news-summarization"
]


# tensorflow_summarization_pipes = [
#     pipeline("summarization", model=model, from_tf=True, device=0, batch_size=4)
#     for model in tensorflow_summarization_models
# ]

In [167]:
text

'DAKWAAN\n  PERTAMA \n- Bahwa Terdakwa KETUT SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada hari Minggu tanggal 02 Juni 2024 sekitar jam 00.30 Wita atau pada suatu waktu dalam bulan Juni 2024 atau setidak-tidaknya dalam tahun 2024 bertempat di pesisir pantai Gondol, Desa Penyabangan, Kec. Gerokgak, Kab. Buleleng, Prov. Bali, atau setidak-tidaknya di suatu tempat yang masih termasuk dalam daerah hukum Pengadilan Negeri Singaraja, dengan sengaja menawarkan atau memberi kesempatan untuk permainan judi yaitu judi bola adil dan menjadikan sebagai pencaharian, atau dengan sengaja turut serta dalam suatu perusahaan untuk itu, yang dilakukan antara lain dengan cara sebagai berikut   -\nBahwa pada waktu dan tempat seperti tersebut diatas, pada saat saksi Polisi Dandik Danny Saputra dan saksi I Made Yogi Mahendra melakukan penyelidikan Pesisir Pantai Gerokgak rawan terjadi adanya tindak pidana, selanjutnya melakukan penangkapan terhadap Terdakwa KETUT SUBAGIA bersama dengan Terdakwa KOMANG A

In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu
import torch
from bert_score import score
from nltk.tokenize import word_tokenize
from nltk.translate import meteor_score
import nltk
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('punkt')

def evaluate_meteor(generated, reference):
    # Tokenize the generated and reference text
    tokenized_generated = word_tokenize(generated)
    tokenized_reference = word_tokenize(reference)
    
    # Evaluate METEOR score
    score = meteor_score.meteor_score([tokenized_reference], tokenized_generated)
    return score

def evaluate_rouge(generated, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return scores

def evaluate_bleu(generated, reference):
    reference_tokens = reference.split()
    generated_tokens = generated.split()
    score = sentence_bleu([reference_tokens], generated_tokens)
    return score

# BERTScore evaluation
def evaluate_bertscore(generated, reference):
    P, R, F1 = score([generated], [reference], lang="en")
    return F1.item()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...


In [194]:
import time
import evaluate

text = df_model.loc[0,'cleaned_dakwaan']
print(len(text))
reference_summary = """Terdakwa Ketut Subagia dan Komang Aditya pada 2 Juni 2024 sekitar pukul 00.30 WITA di pesisir pantai Gondol, Desa Penyabangan, Kecamatan Gerokgak, Kabupaten Buleleng, Bali, diduga mengadakan permainan judi bola adil secara ilegal. Ketut Subagia bertindak sebagai penyelenggara dan pemodal, sementara Komang Aditya membantu sebagai pekerja yang merapikan uang taruhan serta menyerahkannya kepada pemenang. Permainan melibatkan papan bergambar, bola karet, perlak bergambar, dan cek kayu sebagai pengganti uang taruhan. Barang bukti yang disita meliputi papan bola adil, tas, perlak, bola karet, kayu penyeimbang, bedak bayi, cek kayu dalam berbagai denominasi, lap, dan uang tunai Rp1.108.000. Permainan tersebut dilakukan dengan sistem taruhan yang memberikan hadiah kelipatan sembilan kali lipat dari jumlah taruhan jika menang, namun peserta yang kalah kehilangan uangnya. Aktivitas ini berlangsung selama beberapa jam dengan melibatkan sekitar 20 pemain dan keuntungan digunakan untuk kebutuhan sehari-hari. Namun, kegiatan ini dilakukan tanpa izin resmi dan melanggar hukum, sehingga kedua terdakwa ditangkap dan diancam pidana berdasarkan Pasal 303 KUHP juncto Undang-Undang Nomor 7 Tahun 1974 tentang Penertiban Perjudian."""

for i, model in enumerate(pytorch_summarization_models):
    print(model)
    pipe = pipeline("summarization", model=model, device="cuda", batch_size=4, use_fast=False) 
    chunk_size = 512 

    start_time = time.time()

    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    try:
        summaries = [pipe(chunk, max_length=30, min_length=20, do_sample=False)[0]['generated_text'] for chunk in chunks]
    except KeyError:
        summaries = [pipe(chunk, max_length=30, min_length=20, do_sample=False)[0]['summary_text'] for chunk in chunks]

    full_summary = " ".join(summaries)
    end_time = time.time()

    time_taken = end_time - start_time

    print("------------------------------------------------------")

    rouge_scores = evaluate_rouge(full_summary, reference_summary)
    print("ROUGE Scores:", rouge_scores)
    meteor_score_value = evaluate_meteor(full_summary, reference_summary)
    print("METEOR Score:", meteor_score_value)
    bleu_score_value = evaluate_bleu(full_summary, reference_summary)
    print("BLEU Score:", bleu_score_value)
    bertscore_value = evaluate_bertscore(full_summary, reference_summary)
    print("BERTScore F1:", bertscore_value)

    # Display ROUGE scores
    print(len(full_summary))
    print(f"Summary: using {model}: {full_summary}")
    print("------------------------------------------------------")

    del pipe
    torch.cuda.empty_cache()

    print(f"Time taken for summarization: {time_taken:.2f} seconds\n\n")


21776
cahya/t5-base-indonesian-summarization-cased
------------------------------------------------------
ROUGE Scores: {'rouge1': Score(precision=0.11449451887941535, recall=0.5497076023391813, fmeasure=0.18951612903225806), 'rouge2': Score(precision=0.04024390243902439, recall=0.19411764705882353, fmeasure=0.06666666666666667), 'rougeL': Score(precision=0.07429963459196103, recall=0.3567251461988304, fmeasure=0.12298387096774192), 'rougeLsum': Score(precision=0.07429963459196103, recall=0.3567251461988304, fmeasure=0.12298387096774192)}
METEOR Score: 0.2737574220496995
BLEU Score: 0.01669053970928304


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8316876292228699
5476
Summary: using cahya/t5-base-indonesian-summarization-cased: DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada saat terdakwa KETUT SUBAGIA bersama dengan Terdakwa UT SUBAGIA dan Terdakwa KOMANG ADITYA sedang berlangsung permainan perjudian bola adil di pesisir pantai Gondol, Desa Penya Dua buah bola bergambar, satu (satu) buah tas kain warna abu-abu, satu (satu) buah tas karung warna Sebanyak 44 (empat puluh empat) buah papan cek kayu berwarna biru dengan nomor 20 (duapuluh), 17 (tujuh belas) KETUT SUBAGIA dalam permainan judi bola adil tersebut berperan sebagai penyelenggara yang mengadakan permainan judi bola adil dan juga sebagai penyelenggara yang mengadakan permainan KOMANG ADITYA sebagai pekerja dan merapikan uang serta memberikan uang kepada pemenang permainan bola adil. Bahwa dalam permainan judi bola adil Pasangan uang taruhan judi bola adil 1 (satu)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8341520428657532
5481
Summary: using panggi/t5-small-indonesian-summarization-cased: Bahwa Terdakwa KETUT SUBAGIA bersama dengan terdakwa KOMANG ADITYA pada hari Minggu tanggal 02 Juni 2024 udi bola adil dan menjadikan sebagai pencaharian, atau dengan sengaja turut serta dalam suatu perusahaan untuk itu, yang dilakukan antara lain dengan cara sebagai UT SUBAGIA dan Terdakwa KOMANG ADITYA sedang berlangsung permainan perjudian bola adil di pesisir pantai Gondol, Desa Penya bola bergambar, 1 (satu) buah tas kain warna abu-abu. 1 biji biji bola karet warna hitam, 2 biji kacang uh), 44 (empat puluh empat) buah papan cek kayu berwarna biru dengan nomor 20 'duapuluh - duapuluh Rp. 50.000,- (lima puluh ribu rupiah) 4 [empat) lembar uang pecahan Rp yang 20.00, hingga pemodal, mengambil uang - uang taruhan yang berada di atas perlak, menggelindingkan bola, sedangkan Terdakwa KOMANG pan bola adil 1 (satu ) buah perlak bergambar digunakan untuk pasangan uang taruhan judi bola saks

...

BERTScore F1: 0.8033068180084229
176
Summary: using panggi/t5-base-indonesian-summarization-cased:                 Bahwa di papan bola adil terdapat 4 warna yaitu merah, kuning hijau dan hitam adapun gambarnya yaitu berbentuk segitiga, palang, bola,                          
------------------------------------------------------
Time taken for summarization: 17.62 seconds


rayendito/mt5-small-finetuned-xl-sum-indonesia
------------------------------------------------------
ROUGE Scores: {'rouge1': Score(precision=0.13062409288824384, recall=0.5263157894736842, fmeasure=0.20930232558139536), 'rouge2': Score(precision=0.0436046511627907, recall=0.17647058823529413, fmeasure=0.06993006993006994), 'rougeL': Score(precision=0.06676342525399129, recall=0.26900584795321636, fmeasure=0.10697674418604652), 'rougeLsum': Score(precision=0.06676342525399129, recall=0.26900584795321636, fmeasure=0.10697674418604652)}
METEOR Score: 0.2945929244287108
BLEU Score: 0.017863480491804782


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8162375688552856
4457
Summary: using rayendito/mt5-small-finetuned-xl-sum-indonesia: Seorang terdakwa kecelakaan pertama - telah mendakwa seorang terdakwa kecelakaan pertama - setelah Polisi dan saksi Polisi dan saksi Polisi dan saksi Polisi dan saksi Polisi dan saksi Polisi dan Seorang pemain bola adil di Bali telah berlangsung permainan perjudian bola adil di pesisir pantai Gondol, Kec Sebuah tas kain warna hitam, 1 (satu) buah tas kain warna hitam, 1 (satu) buah tas Sebuah lembar uang pecahan Rp. 1.108.000,- (satu juta seratus delapan ribu rupiah) terdiri dari Seorang terdakwa keTUT SUBAGIA dalam permainan judi bola adil di Indonesia, telah mendakwa terdakwa keTUT Suba Pemenang permainan bola adil di Indonesia, telah menentukan kemenangan dalam permainan bola adil, karena seorang pemodal Permainan bola adil 1 (satu) buah tas karung warna biru garis merah digunakan untuk pasangan uang Sebuah papan cek kayu berwarna hitam digunakan untuk penyimpan bola adil setelah dig

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8330612778663635
4361
Summary: using interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs: Bahwa Terdakwa KETUT SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada hari Minggu tanggal 02 Juni 2024 udi bola adil dan menjadikan sebagai pencaharian, atau dengan sengaja turut serta dalam suatu perusahaan UT SUBAGIA dan Terdakwa KOMANG ADITYA sedang berlangsung permainan perjudian bola adil di pesisir pantai buah tas kain warna abu-abu, 1 (satu) buah perlak bergambar, 1 (satu) buah perlak uang pecahan Rp. 1.108.000,- (satu juta seratus delapan ribu rupiah) terdiri dari 8 (delapan Terdakwa KETUT SUBAGIA dalam permainan judi bola adil tersebut berperan sebagai penyelenggara yang mengadakan permainan judi bola Pemodal, mengambil uang - uang taruhan yang berada di atas perlak, menggelindingkan bola, sedangkan PAN bola adil 1 (satu) buah perlak bergambar digunakan untuk pasangan uang taruhan bola adil 1 (satu s kain warna hitam digunakan untuk penyimpan b

In [193]:
import time
import evaluate

text = df_model.loc[0,'cleaned_dakwaan']
print(len(text))
reference_summary = """Terdakwa Ketut Subagia dan Komang Aditya pada 2 Juni 2024 sekitar pukul 00.30 WITA di pesisir pantai Gondol, Desa Penyabangan, Kecamatan Gerokgak, Kabupaten Buleleng, Bali, diduga mengadakan permainan judi bola adil secara ilegal. Ketut Subagia bertindak sebagai penyelenggara dan pemodal, sementara Komang Aditya membantu sebagai pekerja yang merapikan uang taruhan serta menyerahkannya kepada pemenang. Permainan melibatkan papan bergambar, bola karet, perlak bergambar, dan cek kayu sebagai pengganti uang taruhan. Barang bukti yang disita meliputi papan bola adil, tas, perlak, bola karet, kayu penyeimbang, bedak bayi, cek kayu dalam berbagai denominasi, lap, dan uang tunai Rp1.108.000. Permainan tersebut dilakukan dengan sistem taruhan yang memberikan hadiah kelipatan sembilan kali lipat dari jumlah taruhan jika menang, namun peserta yang kalah kehilangan uangnya. Aktivitas ini berlangsung selama beberapa jam dengan melibatkan sekitar 20 pemain dan keuntungan digunakan untuk kebutuhan sehari-hari. Namun, kegiatan ini dilakukan tanpa izin resmi dan melanggar hukum, sehingga kedua terdakwa ditangkap dan diancam pidana berdasarkan Pasal 303 KUHP juncto Undang-Undang Nomor 7 Tahun 1974 tentang Penertiban Perjudian."""

for i, model in enumerate(pytorch_summarization_models):
    print(model)
    pipe = pipeline("summarization", model=model, device="cuda", batch_size=4, use_fast=False) 
    chunk_size = 512 

    start_time = time.time()

    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    try:
        summaries = [pipe(chunk, max_length=60, min_length=40, do_sample=False)[0]['generated_text'] for chunk in chunks]
    except KeyError:
        summaries = [pipe(chunk, max_length=60, min_length=40, do_sample=False)[0]['summary_text'] for chunk in chunks]

    full_summary = " ".join(summaries)
    end_time = time.time()

    time_taken = end_time - start_time

    print("------------------------------------------------------")

    rouge_scores = evaluate_rouge(full_summary, reference_summary)
    print("ROUGE Scores:", rouge_scores)
    meteor_score_value = evaluate_meteor(full_summary, reference_summary)
    print("METEOR Score:", meteor_score_value)
    bleu_score_value = evaluate_bleu(full_summary, reference_summary)
    print("BLEU Score:", bleu_score_value)
    bertscore_value = evaluate_bertscore(full_summary, reference_summary)
    print("BERTScore F1:", bertscore_value)

    # Display ROUGE scores
    print(len(full_summary))
    # print(f"Summary: using {model}: {full_summary}")
    print("------------------------------------------------------")

    del pipe
    torch.cuda.empty_cache()

    print(f"Time taken for summarization: {time_taken:.2f} seconds\n\n")


21776
cahya/t5-base-indonesian-summarization-cased
------------------------------------------------------
ROUGE Scores: {'rouge1': Score(precision=0.0728476821192053, recall=0.7076023391812866, fmeasure=0.13209606986899564), 'rouge2': Score(precision=0.02710843373493976, recall=0.2647058823529412, fmeasure=0.04918032786885246), 'rougeL': Score(precision=0.052378085490668275, recall=0.5087719298245614, fmeasure=0.09497816593886464), 'rougeLsum': Score(precision=0.052378085490668275, recall=0.5087719298245614, fmeasure=0.09497816593886464)}
METEOR Score: 0.24838405513027267
BLEU Score: 0.011384494662711611


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8229100704193115
11014
------------------------------------------------------
Time taken for summarization: 34.53 seconds


panggi/t5-small-indonesian-summarization-cased
------------------------------------------------------
ROUGE Scores: {'rouge1': Score(precision=0.07314285714285715, recall=0.7485380116959064, fmeasure=0.13326392503904216), 'rouge2': Score(precision=0.030303030303030304, recall=0.31176470588235294, fmeasure=0.055237102657634186), 'rougeL': Score(precision=0.05142857142857143, recall=0.5263157894736842, fmeasure=0.09370119729307652), 'rougeLsum': Score(precision=0.05142857142857143, recall=0.5263157894736842, fmeasure=0.09370119729307652)}
METEOR Score: 0.27411036608588657
BLEU Score: 0.012264420925907894


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8367627263069153
11264
------------------------------------------------------
Time taken for summarization: 24.90 seconds


interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch
------------------------------------------------------
ROUGE Scores: {'rouge1': Score(precision=0.08502633559066967, recall=0.6608187134502924, fmeasure=0.15066666666666667), 'rouge2': Score(precision=0.03162650602409638, recall=0.24705882352941178, fmeasure=0.05607476635514018), 'rougeL': Score(precision=0.06019563581640331, recall=0.4678362573099415, fmeasure=0.10666666666666666), 'rougeLsum': Score(precision=0.06019563581640331, recall=0.4678362573099415, fmeasure=0.10666666666666666)}
METEOR Score: 0.256727945764017
BLEU Score: 0.013152770193043332


...

BERTScore F1: 0.8035668730735779
308
------------------------------------------------------
Time taken for summarization: 34.00 seconds


rayendito/mt5-small-finetuned-xl-sum-indonesia
------------------------------------------------------
ROUGE Scores: {'rouge1': Score(precision=0.10062893081761007, recall=0.5614035087719298, fmeasure=0.1706666666666667), 'rouge2': Score(precision=0.033578174186778595, recall=0.18823529411764706, fmeasure=0.05699020480854852), 'rougeL': Score(precision=0.05555555555555555, recall=0.30994152046783624, fmeasure=0.09422222222222223), 'rougeLsum': Score(precision=0.05555555555555555, recall=0.30994152046783624, fmeasure=0.09422222222222223)}
METEOR Score: 0.2726492670718627
BLEU Score: 0.013498977541361845


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8103804588317871
6168
------------------------------------------------------
Time taken for summarization: 23.02 seconds


interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs
------------------------------------------------------
ROUGE Scores: {'rouge1': Score(precision=0.10659439927732611, recall=0.6900584795321637, fmeasure=0.18466353677621283), 'rouge2': Score(precision=0.04159132007233273, recall=0.27058823529411763, fmeasure=0.07210031347962383), 'rougeL': Score(precision=0.07317073170731707, recall=0.47368421052631576, fmeasure=0.12676056338028166), 'rougeLsum': Score(precision=0.07317073170731707, recall=0.47368421052631576, fmeasure=0.12676056338028166)}
METEOR Score: 0.30658265990960154
BLEU Score: 0.018827928157299167


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.845614492893219
7180
------------------------------------------------------
Time taken for summarization: 27.94 seconds




In [119]:
print(len(text))
print(len(full_summary))
print("Summary:", full_summary)

21776
15960
Summary: DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada hari Minggu tanggal 02 Juni 2024 sekitar jam 00.30 Wita atau pada suatu waktu dalam bulan Juni 2024 atau setidak-tidaknya dalam tahun 2024 bertempat di pesisir pantai Gondol, Desa Penyabangan, Kec. Gerokgak, Kab. Buleleng, Prov. Bali, atau setidak- udi bola adil dan menjadikan sebagai pencaharian, atau dengan sengaja turut serta dalam suatu perusahaan untuk itu, yang dilakukan antara lain dengan cara sebagai berikut - Bahwa pada waktu dan tempat seperti tersebut diatas, pada saat saksi Polisi Dandik Danny Saputra dan saksi I Made Yogi Mahendra melakukan penyelidikan Pesisir Pantai Gerokgak rawan terjadi adanya tindak pidana, selanjutnya melakukan penangkapan terhadap Terdakwa KETUT SUBAGIA bersama dengan Terdakwa KOMANG ADITY UT SUBAGIA dan Terdakwa KOMANG ADITYA sedang berlangsung permainan perjudian bola adil di pesisir pantai Gondol, Desa Penyabangan, Kec. Gerokgak, Kab. Bu

ModuleNotFoundError: No module named 'evaluate'